# Preparação do ambiente

Leva de células responsável por preparar todo ambiente deste notebook no Colab para as execuções necessárias para o pré-processamento do corpus de textos.

## Sincronização com Google Drive

Aqui é feita importação de bibliotecas básicas e dos módulos necessários para sincronizar este ambiente (Colab) com o Drive, para obter acesso aos textos extraídos e possibilitar o salvamento dos arquivos pré-processados no Drive.

In [ ]:
import os, string, time
from google.colab import drive
drive.mount('/content/drive')
from google.colab import output
import datetime

def atualizarConsole(nova_string : str) -> None:
  output.clear()
  print(nova_string)

Mounted at /content/drive


## Instalação e importação de bibliotecas que serão utilizadas

In [ ]:
string_exibida = 'Importando statistics, string, os, re, msgpack...\n'
atualizarConsole(string_exibida)
from statistics import mean
import string
import os
import re
!pip install msgpack==1.0.7
import msgpack
string_exibida = '''Importando statistics: OK
Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer...\n'''
atualizarConsole(string_exibida)
try:
  !pip install pt-br-verbs-lemmatizer -U
  from pt_br_verbs_lemmatizer import lemmatize as lematizar, modifyDicAmbiguousVerbs as modificarAmbiguidadeVerbal
except Exception as e:
  print(f'Problema na instalação e importação de pt-br-verbs-lemmatizer:\n{e.__class__.__name__}: {str(e)}')
else:
  string_exibida = '''Importando statistics: OK
Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln...\n'''
  atualizarConsole(string_exibida)
  try:
    !pip install ferramentas-basicas-pln -U
    from ferramentas_basicas_pln import formatarTexto,removerCaracteresEspeciais,removerCaracteresEstranhos,removerEspacosEmBrancoExtras,transformarTextoSubstituindoCaracteres,coletarTextoDeArquivoTxt
    from ferramentas_basicas_pln import STRING_CARACTERES_ESPECIAIS_PADRAO
  except Exception as e:
    print(f'Problema na instalação e importação de ferramentas:\n{e.__class__.__name__}: {str(e)}')
  else:
    string_exibida = '''Importando statistics: OK
Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos...\n'''
    atualizarConsole(string_exibida)
    try:
      import nltk
      from nltk.tokenize import sent_tokenize, word_tokenize
      nltk.download('punkt')
      nltk.download('stopwords')
    except Exception as e:
      print(f'Problema na instalação e importação ou download de módulos de NLTK:\n{e.__class__.__name__}: {str(e)}')
    else:
      string_exibida = '''Importando statistics: OK
Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos: OK
Instalando e importando spaCy e fazendo download de módulos...\n'''
      atualizarConsole(string_exibida)
      try:
        !pip install -U spacy
        !pip install -U spacy-lookups-data
        !python -m spacy download pt_core_news_lg

        import spacy
        # nlp_spacy = spacy.load('pt_core_news_lg')
        nlp_spacy = spacy.load('pt_core_news_lg',enable=["tok2vec","ner","morphologizer"])
        # nlp_spacy.enable_pipe("senter")
      except Exception as e:
        print(f'Problema na instalação e importação ou download de módulos de spaCy:\n{e.__class__.__name__}: {str(e)}')
      else:
        string_exibida = '''Importando statistics: OK
Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos: OK
Instalando e importando spaCy e fazendo download de módulos: OK
'''+'\n'+'-'*100+'\n'+'Ambiente configurado com sucesso.'+'\n'+'-'*100+'\n'
        atualizarConsole(string_exibida)

Importando statistics: OK
Importando string: OK
Importando OS: OK
Importando re: OK
Importando msgpack: OK
Instalando e importando pt-br-verbs-lemmatizer: OK
Instalando e importando ferramentas-basicas-pln: OK
Importando NLTK e fazendo download de módulos: OK
Instalando e importando spaCy e fazendo download de módulos: OK

----------------------------------------------------------------------------------------------------
Ambiente configurado com sucesso.
----------------------------------------------------------------------------------------------------



## Lista de entidades nomeadas personalizada

Além do reconhecimento de entidades no texto proveniente da biblioteca spaCy (para conseguirmos tokenizar mais de uma palavra num único token), estaremos interessados em tokenizar conjuntos de palavras específicos que, provavelmente, o spaCy não entenderá como uma entidade propriamente reconhecida, tais como: "história conceitual", "teoria e filosofia da história", etc...

Nesta etapa da execução estaremos lendo um arquivo de texto que contém tais conjuntos de palavras que serão tokenizadas caso sejam encontradas no texto.

In [ ]:
with open(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Lista entidades nomeadas adição manual.txt','r',encoding='utf-8') as f:
  string_entidades_manuais = f.read()

string_entidades_manuais = ', '.join(string_entidades_manuais.split('\n'))
string_entidades_manuais = removerCaracteresEstranhos(string_entidades_manuais)#.replace('\xa0',' ').replace('\n','')

lista_entidades_nomeadas_manuais = [elemento.lower().strip() for elemento in string_entidades_manuais.split(',')]

lista_entidades_nomeadas_manuais_ordenada = sorted(lista_entidades_nomeadas_manuais, key=len, reverse=True)

lista_entidades_nomeadas_manuais_ordenada

['teoria e filosofia da história',
 'movimento social do contestado',
 'história global do trabalho',
 'história social da cultura',
 'história da historiografia',
 'história de santa catarina',
 'historiografia literária',
 'histórias entrecruzadas',
 'teoria do sistema-mundo',
 'histórias entrelaçadas',
 'história transnacional',
 'história das religiões',
 'história dos conceitos',
 'história contemporânea',
 'filosofia da história',
 'história das mulheres',
 'história do trabalho',
 'guerra do contestado',
 'antiguidade clássica',
 'história conceitual',
 'história conectada',
 'teoria da história',
 'racismo estrutural',
 'história ambiental',
 'história econômica',
 'história cultural',
 'história indígena',
 'estudos de gênero',
 'história medieval',
 'história militar',
 'história da arte',
 'história moderna',
 'história global',
 'história social',
 'teoria marxista',
 'história antiga',
 'sistema mundo',
 'história oral',
 'teoria racial',
 'classe social',
 'teoria queer']

## Lista de stopwords a ser utilizada

Nesta etapa da execução será lido um arquivo de texto que contém as stopwords que estaremos considerando neste pré-processamento/tokenização do corpus.

In [ ]:
with open(os.path.join(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings','Lista de stopwords para pré-processamento - WOKE UFSC.txt'),'r',encoding='utf-8') as f:
  stopwords = f.read()

lista_stopwords = [palavra.strip() for palavra in stopwords.split('\n') if not (palavra.startswith('#') or palavra.startswith('-'))]
print('Lista de stopwords construída sem filtro de repetição, tamanho:',len(lista_stopwords))
print('-'*100)
print('Palavras repetidas encontradas:')
for p in set([elemento for elemento in lista_stopwords if lista_stopwords.count(elemento) > 1]):
  print(p)
print('-'*100)
print('Aplicando filtro...')
print('-'*100)
lista_stopwords = sorted(set([palavra.strip() for palavra in stopwords.split('\n') if not (palavra.startswith('#') or palavra.startswith('-'))]),key=len)
print('Lista de stopwords construída com filtro de repetição, tamanho atualizado:',len(lista_stopwords))

Lista de stopwords construída sem filtro de repetição, tamanho: 238
----------------------------------------------------------------------------------------------------
Palavras repetidas encontradas:
quanta
qual
quantas
quem
aquele
quanto
quantos
vós
nos
que
quais
aquilo
aos
conforme
nós
----------------------------------------------------------------------------------------------------
Aplicando filtro...
----------------------------------------------------------------------------------------------------
Lista de stopwords construída com filtro de repetição, tamanho atualizado: 217


## Modificação da lematização baseada em regras (removendo ambiguidades/duplicidades)

A ferramenta que estaremos utilizando para lematizar os verbos é uma ferramenta baseada em regras. A língua portuguesa, por sem complexa, é comum encontramos diversos empecilhos para trabalhar com PLN baseado em regras (e até mesmo utilizando IA/machine learning). No nosso cenário estas ocasiões, infelizmente, podem ocorrer. Para exemplificar podemos analisar o caso do verbo "foi". Pode ser uma flexão do verbo "ir" ou do verbo "ser": "Ontem eu **fui** na casa do Pedro" e "Eu já **fui** mais cuidadoso". Ambas as frases possuem a mesma flexão verbal fui, porém a lematização nos dois casos será diferente, uma faz referência à "ir" e outra à "ser", respectivamente.

Nesta etapa da execução estaremos modificando (removendo da listagem da ferramenta) verbos no infinitivo que compartilham flexões verbais com outros verbos no infinitivo e flexões verbais que, ao nosso ver, pertencem primariamente à um verbo no infinitivo em específico (dificilmente serão associados à outros).

Detalhando um pouco mais desta etapa da execução, alguns membros do Grupo de Estudos e Pesquisa em IA e História da UFSC ajudaram analisando e escolhendo, manualmente, os verbos referidos a fim de minimizar os erros e otimizar ao máximo a ferramenta para melhor utilização. Na célula de código abaixo podemos ver a separação das escolhas de cada membro e a modificação no processo de lematização que acontecerá na hora de tokenizar os verbos.

In [ ]:
print('Modificando dataset ambiguidades verbais na lematização\nSeleção Mateus...')
modificarAmbiguidadeVerbal(type='infinitive',verbs=['impostar','incender','interditar',
                                                    'interver','irar','iriar','fossar',
                                                    'liar','mantar','mentar','mentirar',
                                                    'podar','possar','preditar',
                                                    'postar','querar','querendar','raer',
                                                    'recobrar','redizer','refugir','remedar',
                                                    'repelar','revir','rer','ruar','segar',
                                                    'erar','seriar','suster'])

modificarAmbiguidadeVerbal(type='flex',verbs=[('relemos','relar'),('relemo-lo','relar'),
                                              ('retemos','retar'),('retemo-lo','retar'),('retemo-nos','retar'),
                                              ('revira','rever'),('reviram','rever'),('reviras','rever'),('revirem','rever'),('revires','rever'),('revira-o','rever'),('revira-lo','rever'),
                                              ('reviste','rever'),('revisto','rever'),('revistes','rever'),('reviste-o','rever'),('revisto-a','rever'),('revisto-o','rever'),('reviste-te','rever'),('revisto-as','rever'),('revisto-me','rever'),('revisto-os','rever'),('revisto-se','rever'),('revisto-te','rever'),
                                              ('revisto-o','rever'),('revisto-me','rever'),
                                              ('ruido-a','ruidar'),('ruido-o','ruidar'),('ruido-as','ruidar'),('ruido-me','ruidar'),('ruido-os','ruidar'),('ruido-se','ruidar'),('ruido-te','ruidar'),
                                              ('sede','ser'),('sede-o','ser'),
                                              ('surta','sortir'),('surte','sortir'),('surto','sortir'),('surtam','sortir'),('surtas','sortir'),('surtem','sortir'),('surtes','sortir'),('surta-o','sortir'),('surtais','sortir'),('surte-o','sortir'),('surto-o','sortir'),('surta-se','sortir'),('surtamos','sortir'),('surte-se','sortir'),('surte-te','sortir'),('surto-me','sortir'),('surtam-no','sortir'),('surtam-se','sortir'),('surtem-no','sortir'),('surtem-se','sortir'),('surtamo-nos','sortir'),
                                              ('some-o','sumir'),('some-se','sumir'),('some-te','sumir'),('somem-no','sumir'),('somem-se','sumir')])
print('Feito')

print('Modificando dataset ambiguidades verbais na lematização\nSeleção Carlos...')
modificarAmbiguidadeVerbal(type='infinitive',verbs=['valar','vestar','vinhar','abolar',
                                                    'açalmar','alardar','amandar','amolecar',
                                                    'apresar','arrefeçar','beberar','bombardar'])

modificarAmbiguidadeVerbal(type='flex',verbs=[('traga','trager'),('trago','trager'),('tragam','trager'),('tragas','trager'),('traga-o','trager'),('tragais','trager'),('trago-o','trager'),('tragamos','trager'),('tragam-no','trager'),
                                              ('abraque','abraçar'),('abracado','abraçar'),('abracará','abraçar'),('abrace-a','abraçar'),('abrace-o','abraçar'),('abraceis','abraçar'),('abraquei','abraçar'),('abraquem','abraçar'),('abraques','abraçar'),('abracai-a','abraçar'),('abracai-o','abraçar'),('abracarás','abraçar'),('abracarão','abraçar'),
                                              ('abracente','abraçar'),('abraque-o','abraçar'),('abraqueis','abraçar'),('abracado-a','abraçar'),('abracado-o','abraçar'),('abracai-as','abraçar'),('abracai-os','abraçar'),
                                              ('abracáveis','abraçar'),('abraquei-o','abraçar'),('abraquemos','abraçar'),('abracado-as','abraçar'),('abracado-la','abraçar'),('abracado-lo','abraçar'),('abracado-me','abraçar'),('abracado-na','abraçar'),('abracado-no','abraçar'),('abracado-os','abraçar'),('abracado-se','abraçar'),('abracado-te','abraçar'),('abracai-lhe','abraçar'),('abracai-vos','abraçar'),('abracaríeis','abraçar'),('abracei-lhe','abraçar'),('abracem-lhe','abraçar'),('abracem-nas','abraçar'),('abracem-nos','abraçar'),('abracemô-la','abraçar'),('abracemô-lo','abraçar'),('abracá-lo-á','abraçar'),('abracáramos','abraçar'),('abracásseis','abraçar'),('abracávamos','abraçar'),('abraquem-no','abraçar'),('abracado-las','abraçar'),('abracado-lhe','abraçar'),('abracado-los','abraçar'),('abracado-nas','abraçar'),('abracado-nos','abraçar'),('abracai-lhes','abraçar'),('abracando-la','abraçar'),('abracando-lo','abraçar'),('abracaríamos','abraçar'),
                                              ('abracá-lo-ei','abraçar'),('abracá-lo-ia','abraçar'),('abracá-lo-ás','abraçar'),('abracá-lo-ão','abraçar'),('abracárei-lo','abraçar'),('abracássemos','abraçar'),('abracávei-lo','abraçar'),('abraquemo-lo','abraçar'),('abracado-lhes','abraçar'),('abracando-las','abraçar'),('abracando-los','abraçar'),('abracemo-lhes','abraçar'),('abracá-lo-eis','abraçar'),('abracá-lo-iam','abraçar'),('abracá-lo-ias','abraçar'),('abracáramo-lo','abraçar'),('abracávamo-lo','abraçar'),('abracá-lo-emos','abraçar'),('abracá-lo-íeis','abraçar'),('abracá-lo-íamos','abraçar'),
                                              ('acaba-a','acaber'),('acaba-o','acaber'),('acabais','acaber'),('acabe-a','acaber'),('acabe-o','acaber'),('acabeis','acaber'),('acabo-a','acaber'),('acabo-o','acaber'),('acaba-as','acaber'),('acaba-me','acaber'),('acaba-os','acaber'),('acaba-se','acaber'),('acaba-te','acaber'),
                                              ('acabe-me','acaber'),('acabe-os','acaber'),('acabe-se','acaber'),('acabe-te','acaber'),('acabei-a','acaber'),('acabei-o','acaber'),
                                              ('acabo-as','acaber'),('acabo-me','acaber'),('acabo-os','acaber'),('acabo-se','acaber'),('acabo-te','acaber'),('acabam-me','acaber'),('acabam-na','acaber'),('acabam-no','acaber'),('acabam-se','acaber'),('acabam-te','acaber'),('acabei-as','acaber'),('acabei-os','acaber'),('acabem-me','acaber'),('acabem-na','acaber'),('acabem-no','acaber'),('acabem-se','acaber'),('acabem-te','acaber'),('acabam-lhe','acaber'),('acabam-nas','acaber'),('acabam-nos','acaber'),('acabamô-la','acaber'),('acabamô-lo','acaber'),('acabei-lhe','acaber'),('acabem-lhe','acaber'),('acabem-nas','acaber'),('acabem-nos','acaber'),('acabemô-la','acaber'),('acabemô-lo','acaber'),('acabam-lhes','acaber'),('acabamo-lhe','acaber'),('acabamo-nas','acaber'),('acabamo-nos','acaber'),('acabamô-las','acaber'),('acabamô-los','acaber'),('acabei-lhes','acaber'),('acabem-lhes','acaber'),('acabemo-lhe','acaber'),('acabemo-nas','acaber'),('acabemo-nos','acaber'),('acabemô-las','acaber'),('acabemô-los','acaber'),('acabamo-lhes','acaber'),('acabemo-lhes','acaber'),
                                              ('achara-a','acharar'),('achara-o','acharar'),('achara-as','acharar'),('achara-os','acharar'),
                                              ('acoitado-me','açoitar'),('acoitáveis','açoitar'),
                                              ('adia','adir'),('adiam','adir'),('adias','adir'),('adia-a','adir'),('adia-o','adir'),('adia-as','adir'),('adia-me','adir'),('adia-os','adir'),('adia-se','adir'),('adia-te','adir'),('adiá-la','adir'),('adiá-lo','adir'),('adia-lhe','adir'),('adiam-me','adir'),('adiam-na','adir'),('adiam-no','adir'),('adiam-se','adir'),('adiam-te','adir'),('adias-me','adir'),('adias-se','adir'),('adias-te','adir'),('adiá-las','adir'),('adiá-los','adir'),('adia-lhes','adir'),('adiam-lhe','adir'),('adiam-nas','adir'),('adiam-nos','adir'),('adiam-lhes','adir'),
                                              ('amente','amar'),
                                              ('assente','assar'),('atente','atar'),
                                              ('atendam-se','atendar'),('atendam-te','atendar'),('atendei-as','atendar'),('atendei-os','atendar'),('atendem-se','atendar'),('atendem-te','atendar'),('atendam-lhe','atendar'),('atendam-nas','atendar'),('atendam-nos','atendar'),('atendamô-la','atendar'),('atendamô-lo','atendar'),('atendei-lhe','atendar'),('atendem-lhe','atendar'),('atendem-nas','atendar'),('atendem-nos','atendar'),('atendemô-la','atendar'),('atendemô-lo','atendar'),('atendam-lhes','atendar'),('atendamo-lhe','atendar'),('atendamo-nas','atendar'),('atendamo-nos','atendar'),('atendamô-las','atendar'),('atendamô-los','atendar'),('atendei-lhes','atendar'),('atendem-lhes','atendar'),('atendemo-lhe','atendar'),('atendemo-nas','atendar'),('atendemo-nos','atendar'),('atendemô-las','atendar'),('atendemô-los','atendar'),('atendamo-lhes','atendar'),('atendemo-lhes','atendar'),
                                              ('avivente','avivar'),
                                              ('calque','calçar'),('calquei','calçar'),('calquem','calçar'),('calques','calçar'),('calquei-o','calçar'),('calquemos','calçar'),
                                              ('valido-se','validar')])
print('Feito')

print('Modificando dataset ambiguidades verbais na lematização\nSeleção Franciele...')
modificarAmbiguidadeVerbal(type='infinitive',verbs=['afeitar','afazendar','aferar',
                                                    'aprazer','aspergir','bular',
                                                    'dardar','denigrir','desmentar',
                                                    'estivar','farar','feitar','feriar',
                                                    'debar','delir','empar','excelir',
                                                    'exir','feder','florar','lambar'])

modificarAmbiguidadeVerbal(type='flex',verbs=[('adira','adir'),('aderem','adir'),('aderes','adir'),('aderia','adir'),('adiram','adir'),('adiras','adir'),('aderiam','adir'),('aderias','adir'),('aderíeis','adir'),('aderíamos','adir'),
                                              ('aposto-se','apostar'),
                                              ('apunhas','apor'),('apunha-o','apor'),
                                              ('avinhas-te','avir'),
                                              ('aviemo-lo','aviar'),
                                              ('caiais','caiar'),
                                              ('compila-o','compelir'),
                                              ('compilo-o','compelir'),
                                              ('composto-te','compostar'),
                                              ('condiríeis','condir'),
                                              ('consumamo-lo','consumar'),
                                              ('contende-lo','conter'),
                                              ('crias-te','crer'),
                                              ('desafeito-os','desafazer'),
                                              ('despeçamo-lo','despedir'),
                                              ('despiste-te','despir'),
                                              ('entrevires','entrevir'),('entrevirdes','entrevir'),
                                              ('giramo-lo','girar'),
                                              ('giro-o','gerir'),
                                              ('carpei','carpar'),('carpa-a','carpar'),('carpe-o','carpar'),('carpeis','carpar'),('carpo-a','carpar'),('carpo-o','carpar'),('carpa-as','carpar'),('carpem-se','carpar'),('carpem-te','carpar'),('carpamô-la','carpar'),('carpamô-lo','carpar'),('carpemô-la','carpar'),('carpemô-lo','carpar'),('carpemo-lhe','carpar'),('carpemo-nas','carpar'),('carpemo-nos','carpar'),('carpemô-las','carpar'),('carpemô-los','carpar'),('carpamo-lhes','carpar'),('carpemo-lhes','carpar'),
                                              ('chovo','chovar'),('chova-a','chovar'),('chova-o','chovar'),('chovais','chovar'),('chove-a','chovar'),('chove-o','chovar'),('chovo-a','chovar'),('chovo-o','chovar'),('chova-as','chovar'),('chova-me','chovar'),('chova-os','chovar'),('chova-se','chovar'),('chova-te','chovar'),('chovamos','chovar'),('chove-as','chovar'),('chove-me','chovar'),('chove-os','chovar'),('chove-se','chovar'),('chove-te','chovar'),('chovei-a','chovar'),('chovei-o','chovar'),('chovemos','chovar'),('chovo-as','chovar'),('chovo-me','chovar'),('chovo-os','chovar'),('chovo-se','chovar'),('chovo-te','chovar'),('chovam-me','chovar'),('chovam-na','chovar'),('chovam-no','chovar'),('chovam-te','chovar'),('chovei-as','chovar'),('chovei-os','chovar'),('chovem-me','chovar'),('chovem-na','chovar'),('chovem-no','chovar'),('chovem-se','chovar'),('chovem-te','chovar'),('chovam-lhe','chovar'),('chovam-nas','chovar'),('chovam-nos','chovar'),('chovamô-la','chovar'),('chovamô-lo','chovar'),('chovei-lhe','chovar'),('chovem-lhe','chovar'),('chovem-nas','chovar'),('chovem-nos','chovar'),('chovemô-la','chovar'),('chovemô-lo','chovar'),('chovam-lhes','chovar'),('chovamo-lhe','chovar'),('chovamo-nas','chovar'),('chovamo-nos','chovar'),('chovamô-las','chovar'),('chovamô-los','chovar'),('chovei-lhes','chovar'),('chovem-lhes','chovar'),('chovemo-lhe','chovar'),('chovemo-nas','chovar'),('chovemo-nos','chovar'),('chovemô-las','chovar'),('chovemô-los','chovar'),('chovamo-lhes','chovar'),('chovemo-lhes','chovar'),
                                              ('cocássemos','coçar'),('cocávei-lo','coçar'),('coquemo-lo','coçar'),('cocado-lhes','coçar'),('cocando-las','coçar'),('cocando-los','coçar'),('cocemo-lhes','coçar'),('cocá-lo-eis','coçar'),('cocá-lo-iam','coçar'),('cocá-lo-ias','coçar'),('cocáramo-lo','coçar'),('cocávamo-lo','coçar'),('cocá-lo-emos','coçar'),('cocá-lo-íeis','coçar'),('cocá-lo-íamos','coçar'),('cocado-las','coçar'),('cocado-lhe','coçar'),('cocado-los','coçar'),('cocado-nas','coçar'),('cocado-nos','coçar'),('cocai-lhes','coçar'),('cocando-la','coçar'),('cocando-lo','coçar'),('cocaríamos','coçar'),('cocáreis','coçar'),('cocáveis','coçar'),('coquei-o','coçar'),('coquemos','coçar'),('cocado-as','coçar'),('cocado-la','coçar'),('cocado-lo','coçar'),('cocado-me','coçar'),('cocado-na','coçar'),('cocado-no','coçar'),('cocado-os','coçar'),('cocado-se','coçar'),('cocado-te','coçar'),('cocai-lhe','coçar'),('cocai-vos','coçar'),('cocaríeis','coçar'),('coque-o','coçar'),('coqueis','coçar'),('cocado-a','coçar'),('cocado-o','coçar'),('cocai-as','coçar'),('cocai-os','coçar'),('coquei','coçar'),('coquem','coçar'),('coques','coçar'),('cocai-a','coçar'),('cocai-o','coçar'),('cocarás','coçar'),('cocarão','coçar'),('coque','coçar'),('cocado','coçar'),('cocará','coçar'),
                                              ('cocei-lhe','cocar'),('cocei-as','cocar'),('cocei-os','cocar'),('coce-as','cocar'),('coce-me','cocar'),('coce-os','cocar'),('coce-se','cocar'),('coce-te','cocar'),('cocei-a','cocar'),('cocei-o','cocar'),('coce-a','cocar'),('coce-o','cocar'),('coces','cocar'),('coce','cocar'),('cocem','cocar'),
                                              ('colige-a','coligar'),('colige-o','coligar'),('coligeis','coligar'),('colige-as','coligar'),('colige-me','coligar'),('colige-os','coligar'),('colige-se','coligar'),('colige-te','coligar'),('coligei-a','coligar'),('coligei-o','coligar'),('coligemos','coligar'),('coligei-as','coligar'),('coligei-os','coligar'),('coligem-me','coligar'),('coligem-na','coligar'),('coligem-no','coligar'),('coligem-se','coligar'),('coligem-te','coligar'),('coligei-lhe','coligar'),('coligem-lhe','coligar'),('coligem-nas','coligar'),('coligem-nos','coligar'),('coligemô-la','coligar'),('coligemô-lo','coligar'),('coligei-lhes','coligar'),('coligem-lhes','coligar'),('coligemo-lhe','coligar'),('coligemo-nas','coligar'),('coligemo-nos','coligar'),('coligemô-las','coligar'),('coligemô-los','coligar'),('coligemo-lhes','coligar'),
                                              ('coloro','colorar'),('coloram','colorar'),('coloras','colorar'),('colora-a','colorar'),('colora-o','colorar'),('colorais','colorar'),('coloreis','colorar'),('coloro-a','colorar'),('coloro-o','colorar'),('colorei-a','colorar'),('colorei-o','colorar'),('coloram-lhe','colorar'),('coloram-nas','colorar'),('coloram-nos','colorar'),('coloramô-la','colorar'),('coloramô-lo','colorar'),('colorei-lhe','colorar'),('coloremô-la','colorar'),('coloremô-lo','colorar'),('coloremo-lhe','colorar'),('coloremo-nas','colorar'),('coloremo-nos','colorar'),('coloremô-las','colorar'),('coloremô-los','colorar'),('coloramo-lhes','colorar'),('coloremo-lhes','colorar'),
                                              ('colorei','colorir'),
                                              ('comediá-la','comedir'),('comediá-lo','comedir'),('comediá-las','comedir'),('comediá-los','comedir'),
                                              ('condesse','condir'),('condessem','condir'),('condesses','condir'),
                                              ('descoloramô-la','descolorar'),('descoloramô-lo','descolorar'),('descoloramô-las','descolorar'),('descoloramô-los','descolorar'),('descoloramo-lhes','descolorar'),('descoloremo-lhes','descolorar'),
                                              ('destrance-se','destrancar'),('destrance-te','destrancar'),
                                              ('destrancárei-lo','destrançar'),('destrancávei-lo','destrançar'),('destrancáramo-lo','destrançar'),('destrancávamo-lo','destrançar'),
                                              ('destroques','destroçar'),('destrocai-a','destroçar'),('destroce-se','destroçar'),('destroce-te','destroçar'),('destrocente','destroçar'),('destroque-o','destroçar'),('destrocado-a','destroçar'),('destrocado-o','destroçar'),('destrocai-as','destroçar'),('destrocai-os','destroçar'),('destrocem-se','destroçar'),('destrocem-te','destroçar'),('destrocado-as','destroçar'),('destrocado-la','destroçar'),('destrocado-lo','destroçar'),('destrocado-me','destroçar'),('destrocado-na','destroçar'),('destrocado-no','destroçar'),('destrocado-os','destroçar'),('destrocei-lhe','destroçar'),('destrocemô-la','destroçar'),('destrocemô-lo','destroçar'),('destrocá-lo-á','destroçar'),('destrocáramos','destroçar'),('destrocásseis','destroçar'),('destrocado-las','destroçar'),('destrocado-lhe','destroçar'),('destrocado-los','destroçar'),('destrocado-nos','destroçar'),('destrocai-lhes','destroçar'),('destrocando-la','destroçar'),('destrocando-lo','destroçar'),('destrocaríamos','destroçar'),('destrocemô-las','destroçar'),('destrocemô-los','destroçar'),('destrocá-lo-ei','destroçar'),('destrocá-lo-ia','destroçar'),('destrocá-lo-ás','destroçar'),('destrocá-lo-ão','destroçar'),('destrocárei-lo','destroçar'),('destrocássemos','destroçar'),('destrocávei-lo','destroçar'),('destroquemo-lo','destroçar'),('destrocado-lhes','destroçar'),('destrocando-las','destroçar'),('destrocando-los','destroçar'),('destrocemo-lhes','destroçar'),('destrocá-lo-eis','destroçar'),('destrocá-lo-iam','destroçar'),('destrocá-lo-ias','destroçar'),('destrocáramo-lo','destroçar'),('destrocávamo-lo','destroçar'),('destrocá-lo-emos','destroçar'),('destrocá-lo-íeis','destroçar'),('destrocá-lo-íamos','destroçar'),
                                              ('dissidia-a','dissidiar'),('dissidias-se','dissidiar'),('dissidias-te','dissidiar'),('dissidiá-las','dissidiar'),('dissidiá-los','dissidiar'),('dissidia-lhes','dissidiar'),
                                              ('doeste-te','doestar'),
                                              ('embebera-a','embeberar'),('embebera-o','embeberar'),('embebera-me','embeberar'),('embebera-os','embeberar'),('embeberá-la','embeberar'),('embeberá-lo','embeberar'),('embeberas-se','embeberar'),('embeberas-te','embeberar'),('embeberem-me','embeberar'),('embeberem-se','embeberar'),('embeberem-te','embeberar'),('embeberá-las','embeberar'),('embeberá-los','embeberar'),
                                              ('empeço-o','empecer'),('empece-as','empecer'),('empece-se','empecer'),('empecem-se','empecer'),('empecemô-los','empecer'),
                                              ('enrice','enriçar'),('enricem','enriçar'),('enrices','enriçar'),('enrique','enriçar'),('enriceis','enriçar'),('enriquei','enriçar'),('enriquem','enriçar'),('enriques','enriçar'),('enricai-a','enriçar'),('enricai-o','enriçar'),('enrice-as','enriçar'),('enrice-se','enriçar'),('enrice-te','enriçar'),('enricei-a','enriçar'),('enricei-o','enriçar'),('enrique-o','enriçar'),('enriqueis','enriçar'),('enricado-a','enriçar'),('enricado-o','enriçar'),('enricai-as','enriçar'),('enricai-os','enriçar'),('enricei-as','enriçar'),('enricei-os','enriçar'),('enricem-se','enriçar'),('enricem-te','enriçar'),('enriquei-o','enriçar'),('enriquemos','enriçar'),('enricado-as','enriçar'),('enricado-la','enriçar'),('enricado-lo','enriçar'),('enricado-me','enriçar'),('enricai-lhe','enriçar'),('enricai-vos','enriçar'),('enricei-lhe','enriçar'),('enricem-lhe','enriçar'),('enricemô-la','enriçar'),('enricemô-lo','enriçar'),('enriquem-no','enriçar'),('enricado-las','enriçar'),('enricado-lhe','enriçar'),('enricado-los','enriçar'),('enricai-lhes','enriçar'),('enricando-la','enriçar'),('enricando-lo','enriçar'),('enricei-lhes','enriçar'),('enricem-lhes','enriçar'),('enricemo-lhe','enriçar'),('enricemô-las','enriçar'),('enricemô-los','enriçar'),('enricárei-lo','enriçar'),('enricávei-lo','enriçar'),('enriquemo-lo','enriçar'),('enricado-lhes','enriçar'),('enricando-las','enriçar'),('enricando-los','enriçar'),('enricemo-lhes','enriçar'),('enricáramo-lo','enriçar'),('enricávamo-lo','enriçar'),
                                              ('entrava-a','entravar'),('entrava-as','entravar'),('entravas-me','entravar'),('entravas-se','entravar'),('entravá-las','entravar'),('entravá-los','entravar'),
                                              ('escure','escurir'),('escures','escurir'),('escuramô-la','escurir'),('escuramô-lo','escurir'),('escurei-lhe','escurir'),('escurem-lhe','escurir'),('escurem-nas','escurir'),('escurem-nos','escurir'),('escuremô-la','escurir'),('escuremô-lo','escurir'),('escuramô-las','escurir'),('escuramô-los','escurir'),('escurei-lhes','escurir'),('escuremô-las','escurir'),('escuremô-los','escurir'),('escuramo-lhes','escurir'),('escuremo-lhes','escurir'),
                                              ('estercaríeis','esterçar'),('estercei-lhe','esterçar'),('estercem-lhe','esterçar'),('estercem-nas','esterçar'),('estercem-nos','esterçar'),('estercemô-la','esterçar'),('estercemô-lo','esterçar'),('estercá-lo-á','esterçar'),('estercáramos','esterçar'),('estercásseis','esterçar'),('estercávamos','esterçar'),('esterquem-no','esterçar'),('estercado-las','esterçar'),('estercado-lhe','esterçar'),('estercado-los','esterçar'),('estercado-nas','esterçar'),('estercado-nos','esterçar'),('estercai-lhes','esterçar'),('estercando-la','esterçar'),('estercando-lo','esterçar'),('estercaríamos','esterçar'),('estercei-lhes','esterçar'),('estercem-lhes','esterçar'),('estercemo-lhe','esterçar'),('estercemo-nas','esterçar'),('estercemo-nos','esterçar'),('estercemô-las','esterçar'),('estercemô-los','esterçar'),('estercá-lo-ei','esterçar'),('estercá-lo-ia','esterçar'),('estercá-lo-ás','esterçar'),('estercá-lo-ão','esterçar'),('estercárei-lo','esterçar'),('estercássemos','esterçar'),('estercávei-lo','esterçar'),('esterquemo-lo','esterçar'),('estercado-lhes','esterçar'),('estercando-las','esterçar'),('estercando-los','esterçar'),('estercemo-lhes','esterçar'),('estercá-lo-eis','esterçar'),('estercá-lo-iam','esterçar'),('estercá-lo-ias','esterçar'),('estercáramo-lo','esterçar'),('estercávamo-lo','esterçar'),('estercá-lo-emos','esterçar'),('estercá-lo-íeis','esterçar'),('estercá-lo-íamos','esterçar'),('estercado-as','esterçar'),('estercado-la','esterçar'),('estercado-lo','esterçar'),('esterquei-o','esterçar'),('estercem-se','esterçar'),('esterce-se','esterçar'),
                                              ('falemô-la','falar'),('falemô-lo','falar'),
                                              ('forque','forçar'),('forquei','forçar'),('forquem','forçar'),('forques','forçar'),('forcai-a','forçar'),('forcai-o','forçar'),('forcarás','forçar'),('forcarão','forçar'),('forque-o','forçar'),('forqueis','forçar'),('forcado-a','forçar'),('forcado-o','forçar'),('forcai-as','forçar'),('forcai-os','forçar'),('forcáreis','forçar'),('forcáveis','forçar'),('forquei-o','forçar'),('forquemos','forçar'),('forcado-as','forçar'),('forcado-la','forçar'),('forcado-lo','forçar'),('forcado-me','forçar'),('forcado-na','forçar'),('forcado-no','forçar'),('forcado-os','forçar'),('forcado-se','forçar'),('forcado-te','forçar'),('forcai-lhe','forçar'),('forcai-vos','forçar'),('forcaríeis','forçar'),('forcei-lhe','forçar'),('forcem-lhe','forçar'),('forcáramos','forçar'),('forcásseis','forçar'),('forcávamos','forçar'),('forquem-no','forçar'),('forcado-las','forçar'),('forcado-lhe','forçar'),('forcado-los','forçar'),('forcado-nas','forçar'),('forcado-nos','forçar'),('forcai-lhes','forçar'),('forcando-la','forçar'),('forcando-lo','forçar'),('forcaríamos','forçar'),('forcá-lo-ei','forçar'),('forcá-lo-ia','forçar'),('forcá-lo-ás','forçar'),('forcá-lo-ão','forçar'),('forcárei-lo','forçar'),('forcássemos','forçar'),('forcávei-lo','forçar'),('forquemo-lo','forçar'),('forcado-lhes','forçar'),('forcando-las','forçar'),('forcando-los','forçar'),('forcá-lo-eis','forçar'),('forcá-lo-iam','forçar'),('forcá-lo-ias','forçar'),('forcáramo-lo','forçar'),('forcávamo-lo','forçar'),('forcá-lo-emos','forçar'),('forcá-lo-íeis','forçar'),('forcá-lo-íamos','forçar'),
                                              ('fumaram-nas','fumarar'),('fumaram-nos','fumarar'),
                                              ('funda-a','fundar'),('fundo-as','fundar'),('fundo-os','fundar'),
                                              ('grasnemo-nas','grasnar'),('grasnemo-nos','grasnar'),('grasnemô-las','grasnar'),('grasnemô-los','grasnar'),('grasnamo-lhes','grasnar'),('grasnemo-lhes','grasnar'),
                                              ('imite-te','imitir'),('imitem-te','imitir'),('imitemô-las','imitir'),('imitemô-los','imitir'),
                                              ('laque','laçar'),('lacado','laçar'),('lacará','laçar'),('laquei','laçar'),('laquem','laçar'),('laques','laçar'),('lacente','laçar'),('laque-o','laçar'),('laqueis','laçar'),('lacado-a','laçar'),('lacado-o','laçar'),('lacáveis','laçar'),('laquei-o','laçar'),('laquemos','laçar'),('lacado-as','laçar'),('lacado-la','laçar'),('lacado-lo','laçar'),('lacado-me','laçar'),('lacásseis','laçar'),('lacávamos','laçar'),('laquem-no','laçar'),('lacáramo-lo','laçar'),('lacávamo-lo','laçar'),('lacá-lo-emos','laçar'),('lacá-lo-íeis','laçar'),('lacá-lo-íamos','laçar'),
                                              ('mascavá-las','mascavar'),('mascavá-los','mascavar'),
                                              ('latei','latir'),('lateu','latir'),('lata-a','latir'),('lata-o','latir'),('latais','latir'),('late-a','latir'),('late-o','latir'),('laterá','latir'),('lati-a','latir'),('lati-o','latir'),('lato-a','latir'),('lato-o','latir'),('lata-as','latir'),('lata-me','latir'),('lata-os','latir'),('lata-se','latir'),('lata-te','latir'),('latei-a','latir'),('latei-o','latir'),('lateres','latir'),('lateria','latir'),('latesse','latir'),('lateste','latir'),('lateu-a','latir'),('lateu-o','latir'),('lati-as','latir'),('lati-me','latir'),('lati-os','latir'),('lati-se','latir'),('lati-te','latir'),('latia-a','latir'),('latia-o','latir'),('lato-as','latir'),('lato-me','latir'),('lato-os','latir'),('lato-se','latir'),('lato-te','latir'),('latê-la','latir'),('latê-lo','latir'),('latíeis','latir'),('latam-me','latir'),('latam-na','latir'),('latam-no','latir'),('late-lhe','latir'),('latei-as','latir'),('latei-os','latir'),('latem-me','latir'),('latem-na','latir'),('latem-no','latir'),('latem-se','latir'),('latem-te','latir'),('later-me','latir'),('later-se','latir'),('later-te','latir'),('latera-a','latir'),('latera-o','latir'),('laterdes','latir'),('lateriam','latir'),('laterias','latir'),('latermos','latir'),('lates-me','latir'),('lates-se','latir'),('lates-te','latir'),('latessem','latir'),('latesses','latir'),('latestes','latir'),('lateu-as','latir'),('lateu-me','latir'),('lateu-os','latir'),('lateu-se','latir'),('lateu-te','latir'),('lateí-la','latir'),('lateí-lo','latir'),('lati-lhe','latir'),('latia-as','latir'),('latia-te','latir'),('latiam-a','latir'),('latiam-o','latir'),('latido-a','latir'),('latido-o','latir'),('latiá-la','latir'),('latiá-lo','latir'),('latê-las','latir'),('latê-los','latir'),('latêreis','latir'),('latamô-la','latir'),('latamô-lo','latir'),('late-lhes','latir'),('latei-lhe','latir'),('latei-vos','latir'),('latem-lhe','latir'),('latem-nas','latir'),('latem-nos','latir'),('latemô-la','latir'),('latemô-lo','latir'),('latera-as','latir'),('latera-me','latir'),('latera-os','latir'),('latera-se','latir'),('latera-te','latir'),('lateremos','latir'),('laterá-la','latir'),('laterá-lo','latir'),('lateríeis','latir'),('lateste-a','latir'),('lateste-o','latir'),('lateu-lhe','latir'),('lateí-las','latir'),('lateí-los','latir'),('lati-lhes','latir'),('latia-lhe','latir'),('latiam-as','latir'),('latido-se','latir'),('latido-te','latir'),('latiá-las','latir'),('latiá-los','latir'),('latê-la-á','latir'),('latê-lo-á','latir'),('latêramos','latir'),('latêsseis','latir'),('latíeí-la','latir'),('latíeí-lo','latir'),('latamô-las','latir'),('latamô-los','latir'),('later-me-á','latir'),('later-se-á','latir'),('later-te-á','latir'),('latera-lhe','latir'),('lateras-te','latir'),('laterem-se','latir'),('laterem-te','latir'),('lateres-me','latir'),('lateres-se','latir'),('lateres-te','latir'),('laterá-las','latir'),('laterá-los','latir'),('lateríamos','latir'),('lateste-as','latir'),('lateste-os','latir'),('latestê-la','latir'),('latestê-lo','latir'),('lateu-lhes','latir'),('latê-la-ei','latir'),('latê-la-ia','latir'),('latê-la-ás','latir'),('latê-la-ão','latir'),('latê-las-á','latir'),('latê-lo-ei','latir'),('latê-lo-ia','latir'),('latê-lo-ás','latir'),('latê-lo-ão','latir'),('latê-los-á','latir'),('latêreí-la','latir'),('latêreí-lo','latir'),('latíeí-las','latir'),('latíeí-los','latir'),('later-se-ás','latir'),('later-se-ão','latir'),('later-te-ei','latir'),('later-te-ia','latir'),('later-te-ás','latir'),('later-te-ão','latir'),('latestes-me','latir'),('latestes-se','latir'),('latestes-te','latir'),('latestê-las','latir'),('latestê-los','latir'),('latiam-lhes','latir'),('latido-lhes','latir'),('latê-la-eis','latir'),('latê-la-iam','latir'),('latê-la-ias','latir'),('latê-las-ei','latir'),('latê-las-ia','latir'),('latê-las-ás','latir'),('latê-las-ão','latir'),('latê-lo-eis','latir'),('latê-lo-iam','latir'),('latê-lo-ias','latir'),('latê-los-ei','latir'),('latê-los-ia','latir'),('latê-los-ás','latir'),('latê-los-ão','latir'),('latêramô-la','latir'),('latêramô-lo','latir'),('latêrei-lhe','latir'),('latêreí-las','latir'),('latêreí-los','latir'),('latíamo-lhe','latir'),('latíamo-nas','latir'),('latíamo-nos','latir'),('latíamô-las','latir'),('latíamô-los','latir'),('latíeis-vos','latir'),('later-me-iam','latir'),('later-me-ias','latir'),('laterdes-vos','latir'),('latermo-lhes','latir'),('lateste-lhes','latir'),('latestes-vos','latir'),('latê-la-emos','latir'),('latê-la-íeis','latir'),('latê-las-eis','latir'),('latê-las-iam','latir'),('latê-las-ias','latir'),('latê-lo-emos','latir'),('latê-lo-íeis','latir'),('latê-los-eis','latir'),('latê-los-iam','latir'),('latê-los-ias','latir'),('latêramo-lhe','latir'),('latêramo-nas','latir'),('latêramo-nos','latir'),('latêramô-las','latir'),('latêramô-los','latir'),('latêrei-lhes','latir'),('latêreis-vos','latir'),('latíamo-lhes','latir'),('later-vos-eis','latir'),('latê-la-íamos','latir'),('latê-las-emos','latir'),('latê-las-íeis','latir'),('latê-lo-íamos','latir'),('latê-los-emos','latir'),('latê-los-íeis','latir'),('latêramo-lhes','latir'),('later-nas-emos','latir'),('later-nos-emos','latir'),('later-vos-íeis','latir'),('latê-las-íamos','latir'),('latê-los-íamos','latir'),('later-lhes-emos','latir'),('later-nas-íamos','latir'),('later-nos-íamos','latir'),('later-lhes-íamos','latir')])
print('Feito')

print('Modificando dataset ambiguidades verbais na lematização\nSeleção Angelo...')
modificarAmbiguidadeVerbal(type='infinitive',verbs=['molir','prendar','presar',
                                                    'quarar','rangir','revenir',
                                                    'soer','transir','vagir',
                                                    'violar'])

modificarAmbiguidadeVerbal(type='flex',verbs=[('moque','moçar'),('mocado','moçar'),('moquei','moçar'),('moquem','moçar'),('moques','moçar'),('mocai-a','moçar'),('mocai-o','moçar'),('moqueis','moçar'),('mocado-a','moçar'),('mocado-o','moçar'),('mocai-as','moçar'),('mocai-os','moçar'),('moquemos','moçar'),('mocado-as','moçar'),('mocado-la','moçar'),('mocado-lo','moçar'),('mocado-me','moçar'),('mocado-na','moçar'),('mocado-no','moçar'),('mocado-os','moçar'),('mocado-se','moçar'),('mocado-te','moçar'),('mocai-lhe','moçar'),('mocai-vos','moçar'),('mocado-las','moçar'),('mocado-lhe','moçar'),('mocado-los','moçar'),('mocado-nas','moçar'),('mocado-nos','moçar'),('mocai-lhes','moçar'),('mocando-la','moçar'),('mocando-lo','moçar'),('mocado-lhes','moçar'),('mocando-las','moçar'),('mocando-los','moçar'),
                                              ('moce','mocar'),('mocem','mocar'),('moces','mocar'),('moce-a','mocar'),('moce-o','mocar'),('moceis','mocar'),('moce-as','mocar'),('moce-me','mocar'),('moce-os','mocar'),('moce-se','mocar'),('moce-te','mocar'),('mocei-a','mocar'),('mocei-o','mocar'),('mocemos','mocar'),('mocente','mocar'),('mocei-as','mocar'),('mocei-os','mocar'),('mocem-me','mocar'),('mocem-na','mocar'),('mocem-no','mocar'),('mocem-se','mocar'),('mocem-te','mocar'),('mocei-lhe','mocar'),('mocem-lhe','mocar'),('mocem-nas','mocar'),('mocem-nos','mocar'),('mocemô-la','mocar'),('mocemô-lo','mocar'),('mocei-lhes','mocar'),('mocem-lhes','mocar'),('mocemo-lhe','mocar'),('mocemo-nas','mocar'),('mocemo-nos','mocar'),('mocemô-las','mocar'),('mocemô-los','mocar'),('mocemo-lhes','mocar'),
                                              ('parente','parar'),
                                              ('ranga','ranger'),('rango','ranger'),('rangam','ranger'),('rangas','ranger'),('ranga-a','ranger'),('ranga-o','ranger'),('rangais','ranger'),('rango-a','ranger'),('rango-o','ranger'),('ranga-as','ranger'),('ranga-me','ranger'),('ranga-os','ranger'),('ranga-se','ranger'),('ranga-te','ranger'),('rangamos','ranger'),('rango-as','ranger'),('rango-me','ranger'),('rango-os','ranger'),('rango-se','ranger'),('rango-te','ranger'),('rangam-me','ranger'),('rangam-na','ranger'),('rangam-no','ranger'),('rangam-se','ranger'),('rangam-te','ranger'),('rangam-lhe','ranger'),('rangam-nas','ranger'),('rangam-nos','ranger'),('rangamô-la','ranger'),('rangamô-lo','ranger'),('rangam-lhes','ranger'),('rangamo-lhe','ranger'),('rangamo-nas','ranger'),('rangamo-nos','ranger'),('rangamô-las','ranger'),('rangamô-los','ranger'),('rangamo-lhes','ranger'),
                                              ('rege','regar'),('rego','regar'),('regei','regar'),('regem','regar'),('reges','regar'),('rege-a','regar'),('rege-o','regar'),('regeis','regar'),('rege-as','regar'),('rege-me','regar'),('rege-os','regar'),('rege-se','regar'),('rege-te','regar'),('regei-a','regar'),('regei-o','regar'),('regemos','regar'),('regei-as','regar'),('regei-os','regar'),('regem-me','regar'),('regem-na','regar'),('regem-no','regar'),('regem-se','regar'),('regem-te','regar'),('regem-lhe','regar'),('regem-nas','regar'),('regem-nos','regar'),('regemô-la','regar'),('regemô-lo','regar'),('regei-lhes','regar'),('regem-lhes','regar'),('regemo-lhe','regar'),('regemo-nas','regar'),('regemo-nos','regar'),('regemô-las','regar'),('regemô-los','regar'),('regemo-lhes','regar'),
                                              ('rega','reger'),('regam','reger'),('regas','reger'),('rega-a','reger'),('rega-o','reger'),('regais','reger'),('rego-a','reger'),('rego-o','reger'),('rega-as','reger'),('rega-me','reger'),('rega-os','reger'),('rega-se','reger'),('rega-te','reger'),('regamos','reger'),('rego-as','reger'),('rego-me','reger'),('rego-os','reger'),('rego-se','reger'),('rego-te','reger'),('regam-me','reger'),('regam-na','reger'),('regam-no','reger'),('regam-se','reger'),('regam-te','reger'),('regam-lhe','reger'),('regam-nas','reger'),('regam-nos','reger'),('regamô-la','reger'),('regamô-lo','reger'),('regam-lhes','reger'),('regamo-lhe','reger'),('regamo-nas','reger'),('regamo-nos','reger'),('regamô-las','reger'),('regamô-los','reger'),('regamo-lhes','reger'),
                                              ('regerá-la','reger'),('regerá-lo','reger'),
                                              ('regravá-la','regrar'),('regravá-lo','regrar'),
                                              ('relavá-la','relar',),('relavá-lo','relar'),('relavá-las','relar'),('relente','relar'),
                                              ('rema','remir'),('reme','remir'),('remo','remir'),('remam','remir'),('remas','remir'),('remei','remir'),('remem','remir'),('remes','remir'),('rema-a','remir'),('rema-o','remir'),('remais','remir'),('reme-a','remir'),('reme-o','remir'),('remeis','remir'),('remo-a','remir'),('remo-o','remir'),('rema-as','remir'),('rema-me','remir'),('rema-os','remir'),('rema-se','remir'),('rema-te','remir'),('remamos','remir'),('reme-as','remir'),('reme-me','remir'),('reme-os','remir'),('reme-se','remir'),('reme-te','remir'),('remei-a','remir'),('remei-o','remir'),('rememos','remir'),('remo-as','remir'),('remo-me','remir'),('remo-os','remir'),('remo-se','remir'),('remo-te','remir'),('remam-me','remir'),('remam-na','remir'),('remam-no','remir'),('remam-se','remir'),('remam-te','remir'),('remei-as','remir'),('remei-os','remir'),('remem-me','remir'),('remem-na','remir'),('remem-no','remir'),('remem-se','remir'),('remem-te','remir'),('remam-lhe','remir'),('remam-nas','remir'),('remam-nos','remir'),('remamô-la','remir'),('remamô-lo','remir'),('remei-lhe','remir'),('remem-lhe','remir'),('remem-nas','remir'),('remem-nos','remir'),('rememô-la','remir'),('rememô-lo','remir'),('remam-lhes','remir'),('remamo-lhe','remir'),('remamo-nas','remir'),('remamo-nos','remir'),('remamô-las','remir'),('remamô-los','remir'),('remei-lhes','remir'),('remem-lhes','remir'),('rememo-lhe','remir'),('rememo-nas','remir'),('rememo-nos','remir'),('rememô-las','remir'),('rememô-los','remir'),('remamo-lhes','remir'),('rememo-lhes','remir'),('remesse','remir'),('remessem','remir'),('remesses','remir'),
                                              ('represente','represar'),
                                              ('retente','retar'),
                                              ('roce','rocar'),('rocem','rocar'),('roces','rocar'),('roce-a','rocar'),('roce-o','rocar'),('roceis','rocar'),('roce-as','rocar'),('roce-me','rocar'),('roce-os','rocar'),('roce-se','rocar'),('roce-te','rocar'),('rocei-a','rocar'),('rocei-o','rocar'),('rocemos','rocar'),('rocente','rocar'),('rocai-as','rocar'),('rocai-os','rocar'),('rocei-as','rocar'),('rocei-os','rocar'),('rocem-me','rocar'),('rocem-na','rocar'),('rocem-no','rocar'),('rocem-se','rocar'),('rocem-te','rocar'),('rocei-lhe','rocar'),('rocem-lhe','rocar'),('rocem-nas','rocar'),('rocem-nos','rocar'),('rocemô-la','rocar'),('rocemô-lo','rocar'),('rocei-lhes','rocar'),('rocem-lhes','rocar'),('rocemo-lhe','rocar'),('rocemo-nas','rocar'),('rocemo-nos','rocar'),('rocemô-las','rocar'),('rocemô-los','rocar'),('rocemo-lhes','rocar'),
                                              ('roque','roçar'),('rocado','roçar'),('rocará','roçar'),('roquei','roçar'),('roquem','roçar'),('roques','roçar'),('rocai-a','roçar'),('rocai-o','roçar'),('rocarás','roçar'),('rocarão','roçar'),('roqueis','roçar'),('rocado-a','roçar'),('rocado-o','roçar'),('rocáreis','roçar'),('rocáveis','roçar'),('roquemos','roçar'),('rocado-as','roçar'),('rocado-la','roçar'),('rocado-lo','roçar'),('rocado-me','roçar'),('rocado-na','roçar'),('rocado-no','roçar'),('rocado-os','roçar'),('rocado-se','roçar'),('rocado-te','roçar'),('rocai-lhe','roçar'),('rocai-vos','roçar'),('rocaríeis','roçar'),('rocáramos','roçar'),('rocásseis','roçar'),('rocávamos','roçar'),('rocado-las','roçar'),('rocado-lhe','roçar'),('rocado-los','roçar'),('rocado-nas','roçar'),('rocado-nos','roçar'),('rocai-lhes','roçar'),('rocando-la','roçar'),('rocando-lo','roçar'),('rocaríamos','roçar'),('rocássemos','roçar'),('rocado-lhes','roçar'),('rocando-las','roçar'),('rocando-los','roçar'),
                                              ('ruge','rugar'),('rugem','rugar'),('ruges','rugar'),('ruge-a','rugar'),('ruge-o','rugar'),('rugeis','rugar'),('ruge-as','rugar'),('ruge-me','rugar'),('ruge-os','rugar'),('ruge-se','rugar'),('ruge-te','rugar'),('rugei-a','rugar'),('rugei-o','rugar'),('rugemos','rugar'),('rugei-as','rugar'),('rugei-os','rugar'),('rugem-me','rugar'),('rugem-na','rugar'),('rugem-no','rugar'),('rugem-se','rugar'),('rugem-te','rugar'),('rugei-lhe','rugar'),('rugem-lhe','rugar'),('rugem-nas','rugar'),('rugem-nos','rugar'),('rugemô-la','rugar'),('rugemô-lo','rugar'),('rugei-lhes','rugar'),('rugem-lhes','rugar'),('rugemo-lhe','rugar'),('rugemo-nas','rugar'),('rugemo-nos','rugar'),('rugemô-las','rugar'),('rugemô-los','rugar'),('rugemo-lhes','rugar'),
                                              ('ruga','rugir'),('rugo','rugir'),('rugam','rugir'),('rugas','rugir'),('ruga-a','rugir'),('ruga-o','rugir'),('rugais','rugir'),('rugo-a','rugir'),('rugo-o','rugir'),('ruga-as','rugir'),('ruga-me','rugir'),('ruga-os','rugir'),('ruga-se','rugir'),('ruga-te','rugir'),('rugamos','rugir'),('rugo-as','rugir'),('rugo-me','rugir'),('rugo-os','rugir'),('rugo-se','rugir'),('rugo-te','rugir'),('rugam-me','rugir'),('rugam-na','rugir'),('rugam-no','rugir'),('rugam-se','rugir'),('rugam-te','rugir'),('rugam-lhe','rugir'),('rugam-nas','rugir'),('rugam-nos','rugir'),('rugamô-la','rugir'),('rugamô-lo','rugir'),('rugam-lhes','rugir'),('rugamo-lhe','rugir'),('rugamo-nas','rugir'),('rugamo-nos','rugir'),('rugamô-las','rugir'),('rugamô-los','rugir'),('rugamo-lhes','rugir'),
                                              ('surtei','surtir'),('surteis','surtir'),('surtei-a','surtir'),('surtei-o','surtir'),('surtei-as','surtir'),('surtei-os','surtir'),('surtei-lhe','surtir'),('surtei-lhes','surtir'),
                                              ('sustente','sustar'),
                                              ('trage','tragar'),('tragei','tragar'),('tragem','tragar'),('trages','tragar'),('trage-a','tragar'),('trage-o','tragar'),('trageis','tragar'),('trage-as','tragar'),('trage-me','tragar'),('trage-os','tragar'),('trage-se','tragar'),('trage-te','tragar'),('tragei-a','tragar'),('tragei-o','tragar'),('tragemos','tragar'),('tragei-as','tragar'),('tragei-os','tragar'),('tragem-me','tragar'),('tragem-na','tragar'),('tragem-no','tragar'),('tragem-se','tragar'),('tragem-te','tragar'),('tragei-lhe','tragar'),('tragem-lhe','tragar'),('tragem-nas','tragar'),('tragem-nos','tragar'),('tragemô-la','tragar'),('tragemô-lo','tragar'),('tragei-lhes','tragar'),('tragem-lhes','tragar'),('tragemo-lhe','tragar'),('tragemo-nas','tragar'),('tragemo-nos','tragar'),('tragemô-las','tragar'),('tragemô-los','tragar'),('tragemo-lhes','tragar'),
                                              ('traga','trager'),('trago','trager'),('tragam','trager'),('tragas','trager'),('traga-a','trager'),('traga-o','trager'),('tragais','trager'),('trago-a','trager'),('trago-o','trager'),('traga-as','trager'),('traga-me','trager'),('traga-os','trager'),('traga-se','trager'),('traga-te','trager'),('tragamos','trager'),('trago-as','trager'),('trago-me','trager'),('trago-os','trager'),('trago-se','trager'),('trago-te','trager'),('tragam-me','trager'),('tragam-na','trager'),('tragam-no','trager'),('tragam-se','trager'),('tragam-te','trager'),('tragam-lhe','trager'),('tragam-nas','trager'),('tragam-nos','trager'),('tragamô-la','trager'),('tragamô-lo','trager'),('tragam-lhes','trager'),('tragamo-lhe','trager'),('tragamo-nas','trager'),('tragamo-nos','trager'),('tragamô-las','trager'),('tragamô-los','trager'),('tragamo-lhes','trager'),
                                              ('trance','trancar'),('trancem','trancar'),('trances','trancar'),('trance-a','trancar'),('trance-o','trancar'),('tranceis','trancar'),('trance-as','trancar'),('trance-me','trancar'),('trance-os','trancar'),('trance-se','trancar'),('trance-te','trancar'),('trancei-a','trancar'),('trancei-o','trancar'),('trancemos','trancar'),('trancente','trancar'),('trancei-as','trancar'),('trancei-os','trancar'),('trancem-me','trancar'),('trancem-na','trancar'),('trancem-no','trancar'),('trancem-se','trancar'),('trancem-te','trancar'),('trancei-lhe','trancar'),('trancem-lhe','trancar'),('trancem-nas','trancar'),('trancem-nos','trancar'),('trancemô-la','trancar'),('trancemô-lo','trancar'),('trancei-lhes','trancar'),('trancem-lhes','trancar'),('trancemo-lhe','trancar'),('trancemo-nas','trancar'),('trancemo-nos','trancar'),('trancemô-las','trancar'),('trancemô-los','trancar'),('trancemo-lhes','trancar'),
                                              ('tranque','trançar'),('trancado','trançar'),('trancará','trançar'),('tranquei','trançar'),('tranquem','trançar'),('tranques','trançar'),('trancai-a','trançar'),('trancai-o','trançar'),('trancarás','trançar'),('trancarão','trançar'),('tranque-o','trançar'),('tranqueis','trançar'),('trancado-a','trançar'),('trancado-o','trançar'),('trancai-as','trançar'),('trancai-os','trançar'),('trancáreis','trançar'),('trancáveis','trançar'),('tranque-se','trançar'),('tranquei-o','trançar'),('tranquemos','trançar'),('trancado-as','trançar'),('trancado-la','trançar'),('trancado-lo','trançar'),('trancado-me','trançar'),('trancado-na','trançar'),('trancado-no','trançar'),('trancado-os','trançar'),('trancado-se','trançar'),('trancado-te','trançar'),('trancai-lhe','trançar'),('trancai-vos','trançar'),('trancaríeis','trançar'),('trancá-lo-á','trançar'),('trancáramos','trançar'),('trancásseis','trançar'),('trancávamos','trançar'),('tranquei-me','trançar'),('tranquem-no','trançar'),('tranquem-se','trançar'),('trancado-las','trançar'),('trancado-lhe','trançar'),('trancado-los','trançar'),('trancado-nas','trançar'),('trancado-nos','trançar'),('trancai-lhes','trançar'),('trancando-la','trançar'),('trancando-lo','trançar'),('trancar-se-á','trançar'),('trancaríamos','trançar'),('trancaste-te','trançar'),('trancá-lo-ei','trançar'),('trancá-lo-ia','trançar'),('trancá-lo-ás','trançar'),('trancá-lo-ão','trançar'),('trancárei-lo','trançar'),('trancássemos','trançar'),('trancávei-lo','trançar'),('tranquemo-lo','trançar'),('trancado-lhes','trançar'),('trancando-las','trançar'),('trancando-los','trançar'),('trancar-se-ão','trançar'),('trancar-te-ás','trançar'),('trancá-lo-eis','trançar'),('trancá-lo-iam','trançar'),('trancá-lo-ias','trançar'),('trancáramo-lo','trançar'),('trancávamo-lo','trançar'),('tranquemo-nos','trançar'),('trancá-lo-emos','trançar'),('trancá-lo-íeis','trançar'),('trancáramo-nos','trançar'),('trancáreis-vos','trançar'),('trancávamo-nos','trançar'),('trancáveis-vos','trançar'),('trancá-lo-íamos','trançar'),('trancar-vos-íeis','trançar'),('trancar-nos-íamos','trançar'),
                                              ('troce','trocar'),('trocem','trocar'),('troces','trocar'),('troce-a','trocar'),('troce-o','trocar'),('troceis','trocar'),('troce-as','trocar'),('troce-me','trocar'),('troce-os','trocar'),('troce-se','trocar'),('troce-te','trocar'),('trocei-a','trocar'),('trocei-o','trocar'),('trocemos','trocar'),('trocente','trocar'),('trocei-as','trocar'),('trocei-os','trocar'),('trocem-me','trocar'),('trocem-na','trocar'),('trocem-no','trocar'),('trocem-se','trocar'),('trocem-te','trocar'),('trocei-lhe','trocar'),('trocem-lhe','trocar'),('trocem-nas','trocar'),('trocem-nos','trocar'),('trocemô-la','trocar'),('trocemô-lo','trocar'),('trocei-lhes','trocar'),('trocem-lhes','trocar'),('trocemo-lhe','trocar'),('trocemo-nas','trocar'),('trocemo-nos','trocar'),('trocemô-las','trocar'),('trocemô-los','trocar'),('trocemo-lhes','trocar'),
                                              ('troque','troçar'),('trocado','troçar'),('trocará','troçar'),('troquei','troçar'),('troquem','troçar'),('troques','troçar'),('trocai-a','troçar'),('trocai-o','troçar'),('trocarás','troçar'),('trocarão','troçar'),('troque-o','troçar'),('troqueis','troçar'),('trocado-a','troçar'),('trocado-o','troçar'),('trocai-as','troçar'),('trocai-os','troçar'),('trocáreis','troçar'),('trocáveis','troçar'),('troque-se','troçar'),('troquei-o','troçar'),('troquemos','troçar'),('trocado-as','troçar'),('trocado-la','troçar'),('trocado-lo','troçar'),('trocado-me','troçar'),('trocado-na','troçar'),('trocado-no','troçar'),('trocado-os','troçar'),('trocado-se','troçar'),('trocado-te','troçar'),('trocai-lhe','troçar'),('trocai-vos','troçar'),('trocaríeis','troçar'),('trocá-lo-á','troçar'),('trocáramos','troçar'),('trocásseis','troçar'),('trocávamos','troçar'),('troquei-me','troçar'),('troquem-no','troçar'),('troquem-se','troçar'),('trocado-las','troçar'),('trocado-lhe','troçar'),('trocado-los','troçar'),('trocado-nas','troçar'),('trocado-nos','troçar'),('trocai-lhes','troçar'),('trocando-la','troçar'),('trocando-lo','troçar'),('trocar-se-á','troçar'),('trocaríamos','troçar'),('trocaste-te','troçar'),('trocá-lo-ei','troçar'),('trocá-lo-ia','troçar'),('trocá-lo-ás','troçar'),('trocá-lo-ão','troçar'),('trocárei-lo','troçar'),('trocássemos','troçar'),('trocávei-lo','troçar'),('troquemo-lo','troçar'),('trocado-lhes','troçar'),('trocando-las','troçar'),('trocando-los','troçar'),('trocar-se-ão','troçar'),('trocar-te-ás','troçar'),('trocá-lo-eis','troçar'),('trocá-lo-iam','troçar'),('trocá-lo-ias','troçar'),('trocáramo-lo','troçar'),('trocávamo-lo','troçar'),('troquemo-nos','troçar'),('trocá-lo-emos','troçar'),('trocá-lo-íeis','troçar'),('trocáramo-nos','troçar'),('trocáreis-vos','troçar'),('trocávamo-nos','troçar'),('trocáveis-vos','troçar'),('trocá-lo-íamos','troçar'),('trocar-vos-íeis','troçar'),('trocar-nos-íamos','troçar'),
                                              ('vivente','vivar'),
                                              ('viga','viger'),('vigo','viger'),('vigam','viger'),('vigas','viger'),('viga-a','viger'),('viga-o','viger'),('vigais','viger'),( 'vigo-a','viger'),('vigo-o','viger'),('viga-as','viger'),('viga-me','viger'),('viga-os','viger'),('viga-se','viger'),('viga-te','viger'),('vigamos','viger'),('vigo-as','viger'),('vigo-me','viger'),('vigo-os','viger'),('vigo-se','viger'),('vigo-te','viger'),('vigam-me','viger'),('vigam-na','viger'),('vigam-no','viger'),('vigam-se','viger'),('vigam-te','viger'),('vigam-lhe','viger'),('vigam-nas','viger'),('vigam-nos','viger'),('vigamô-la','viger'),('vigamô-lo','viger'),('vigam-lhes','viger'),('vigamo-lhe','viger'),('vigamo-nas','viger'),('vigamo-nos','viger'),('vigamô-las','viger'),('vigamô-los','viger'),('vigamo-lhes','viger'),
                                              ('vige','vigar'),('vigei','vigar'),('vigem','vigar'),('viges','vigar'),('vige-a','vigar'),('vige-o','vigar'),('vigeis','vigar'),('vige-as','vigar'),('vige-me','vigar'),('vige-os','vigar'),('vige-se','vigar'),('vige-te','vigar'),('vigei-a','vigar'),('vigei-o','vigar'),('vigemos','vigar'),('vigei-as','vigar'),('vigei-os','vigar'),('vigem-me','vigar'),('vigem-na','vigar'),('vigem-no','vigar'),('vigem-se','vigar'),('vigem-te','vigar'),('vigei-lhe','vigar'),('vigem-lhe','vigar'),('vigem-nas','vigar'),('vigem-nos','vigar'),('vigemô-la','vigar'),('vigemô-lo','vigar'),('vigei-lhes','vigar'),('vigem-lhes','vigar'),('vigemo-lhe','vigar'),('vigemo-nas','vigar'),('vigemo-nos','vigar'),('vigemô-las','vigar'),('vigemô-los','vigar'),('vigemo-lhes','vigar')])
print('Feito')

Modificando dataset ambiguidades verbais na lematização
Seleção Mateus...
Feito
Modificando dataset ambiguidades verbais na lematização
Seleção Carlos...
Feito
Modificando dataset ambiguidades verbais na lematização
Seleção Franciele...
Feito
Modificando dataset ambiguidades verbais na lematização
Seleção Angelo...
Feito


## Funções

In [ ]:
mascara_link = 'mascara_link_url' # Toda vez que for encontrado um link, na hora de tokenizá-lo o token será substituído pelo valor da variável "mascara_link"
mascara_email = 'mascara_email' # Toda vez que for encontrado um email, na hora de tokenizá-lo o token será substituído pelo valor da variável "mascara_email"
mascara_dinheiro = 'R$' # Toda vez que for encontrado um valor em dinheiro, na hora de tokenizá-lo o valor em dinheiro será tokenizado com o valor da variável "mascara_dinheiro"

def padronizacaoTexto(texto : str) -> str:
  """
  Função responsável por realizar uma padronização do texto, executando algumas
  formatações.
  Aqui optou-se por formatar da seguinte forma:
  - Remover caractéres estranhos (que não possuem no teclado, basicamente);
  - Padronizar os espaços em branco (se estiver espaços em branco a mais será
  padronizado para apenas um);
  - Padronizar, com a máscara, links;
  - Padronizar, com a máscara, emails;
  - Padronizar, com a máscara, valores de dinheiro;

  Para mais informação sobre como é feito a formatação, visitar o link da biblioteca que foi
  importada a função "formatarTexto()" disponível em: https://pypi.org/project/ferramentas-basicas-pln/

   ### Parâmetros:
  - texto: String contendo o texto que se deseja padronizar/formatar.

  ### Retornos:
  - String contendo o texto formatado.
  """
  texto = texto.replace('\n',' ').replace('ü','u')
  return formatarTexto(texto=texto,
                       padronizar_texto_para_minuscula=False,
                       remover_caracteres_especiais=False,
                       remover_caracteres_estranhos=True,
                       remover_espacos_em_branco_em_excesso=True,
                       padronizar_links=True,padrao_link=mascara_link,
                       padronizar_dinheiros=True,padrao_dinheiro=mascara_dinheiro,
                       padronizar_emails=True,padrao_email=mascara_email)

def padronizacaoTokens(lista_frases_tokenizadas : list[str],
                       lista_tokens_personalizados : list[str] = []) -> list[str]:
  """
  Função responsável por padronizar os tokens antes da finalização da sua tokenização

  ### Parâmetros:
  - lista_frases_tokenizadas: variável do tipo list com os elementos do tipo
  list contendo strings (tokens) dentro.
  Ou seja, uma lista de listas de tokens.
  - lista_tokens_personalizados: variável do tipo list com os elementos do
  tipo str que contém os tokens que não deverão passar por nenhuma formatação.

  ### Retornos:
  - String contendo o token formatado.
  """
  lista_frases_tokenizadas_padronizada = []
  lista_tokens_personalizados += [mascara_link,mascara_dinheiro,mascara_email]

  for frase in lista_frases_tokenizadas:
    lista_frases_tokenizadas_padronizada_atual = []
    for token in frase:
      if (token not in STRING_CARACTERES_ESPECIAIS_PADRAO) and (token.lower() not in lista_stopwords): # Caso o token não esteja na lista de stopwords e não seja um caracter especial
        if token not in lista_tokens_personalizados: # Caso o token não esteja na lista de tokens personalizados (que não serão formatados)
          # Formatação removendo caractéres especiais e passando todas as letras para minúscula além de remover possíveis espaços em branco nas extremidades
          token = formatarTexto(texto=token,
                                remover_caracteres_especiais=True,
                                remover_caracteres_estranhos=False,
                                remover_espacos_em_branco_em_excesso=False,
                                padronizar_texto_para_minuscula=True).strip()
        else: # Se for token personalizado apenas remover, caso possua, os espaços em branco nas extremidades
          token = token.strip()

        if len(token) > 1: # Se o token tiver mais de uma letra ele será tokenizado e adicionado à frase formatada, se não será excluído do corpus pré-processado
          lista_frases_tokenizadas_padronizada_atual.append(token)


    if lista_frases_tokenizadas_padronizada_atual: # Se a lista de tokens na frase não estiver vazia, adicionar na lista de listas de frases
      lista_frases_tokenizadas_padronizada.append(lista_frases_tokenizadas_padronizada_atual)

  if lista_frases_tokenizadas_padronizada: # Se a lista de listas de frases não estiver vazia
    return lista_frases_tokenizadas_padronizada

def substituirEntidadeNomeadaManual(texto : str,
                                    entidade_normal : str,
                                    entidade_formatada : str) -> str:
  """
  Função responsável por formatar a entidade nomeada encontrada numa frase.

  ### Parâmetros:
  - texto: variável do tipo string contendo a frase que terá sua entidade em
  questão formatada.
  - entidade_normal: variável do tipo string que conterá a entidade a ser formatada.
  - entidade_formatada: variável do tipo string que conterá a entidade formatada
  a ser substituída na frase.

  ### Retornos:
  - String contendo a frase com a entidade em questão formatada.
  """
  return re.sub(r'{x}'.format(x=entidade_normal), entidade_formatada, texto, flags=re.IGNORECASE)


def salvarArquivoMsgPack(nome_variavel : str,
                         variavel_em_questao,
                         pasta_para_salvar : str = '') -> tuple[bool,str]:
    """
    Função responsável por salvar, no formato "msgpack", alguma variável deste
    ambiente numa pasta especificada.

    ### Parâmetros:
    -

    ### Retornos:
    - Tupla com dois elementos. O primeiro do tipo bool que faz referência ao
    status do processo (True bem sucedido, False falhou) e o segundo do tipo
    string contendo a mensagem referente ao status obtido.
    """
    try:
        # Se a pasta não existir, efetuamos sua criação
        if not os.path.exists(pasta_para_salvar):
            os.makedirs(pasta_para_salvar)

        # Obtendo os bytes da variável que queremos salvar
        variable_bytes = msgpack.packb(variavel_em_questao)

        # Se o nome do arquivo que salvará a variável não tiver o formato msgpack ao final, iremos adicioná-lo.
        if not nome_variavel.endswith('.msgpack'):
            nome_variavel += '.msgpack'

        # Salvando a variável desejada em bytes no formato .msgpack
        with open(os.path.join(pasta_para_salvar,nome_variavel),'wb') as f:
            f.write(variable_bytes)
            f.close()
            return True, f'Variável {nome_variavel} salva com sucesso no formato .msgpack'

    # Caso ocorra algum erro inesperado, trataremos o enviando como mensagem de falha juntamente com um status False (de falha/erro no processo)
    except Exception as e:
        error_message = f'{e.__class__.__name__}: {str(e)}'
        return False, error_message

def abrirArquivoMsgPack(full_filepath : str,
                        encoding_type : str = None):
    """
    Função responsável por abrir os arquivos no formato msgpack.

    ### Parâmetros:
    - full_filepath: String contendo o caminho completo até o arquivo que deseja-se
    abrir e extrair o conteúdo (variável salva).
    - encoding_type: String contendo o tipo de encoding, caso desejar.

    ### Retornos:
    - Variável salva (e agora aberta e lida) no arquivo msgpack.
    """
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded


def checarQuantidadePalavrasNaFrase(frase : str,
                                    quantidade_minima_de_palavras : int = 5) -> bool:
  """
  Função responsável por checar a quantidade de palavras presentes numa frase.

  ### Parâmetros:
  - frase: String contendo a frase a ser analisada.
  - quantidade_minima_de_palavras: Inteiro que faz referência à quantidade mínima
  de palavras aceitável para a frase entrar pro corpus pré-processado.

  ### Retoros:
  - Bool que alimentará a escolha de adicionar a frase ao corpus pré-processado
  (True) ou não (False).
  """
  if len(removerCaracteresEspeciais(frase).split()) >= quantidade_minima_de_palavras:
    return True
  return False

def preProcessarTextoSemRE(texto : str,
                           caminho_para_salvar_tokenizacao : str,
                           silence : bool = True) -> bool:
  """
  Função responsável por pré-processar o texto sem a tokenização usando o
  reconhecimento de entidades (tokenizar entidades nomeadas). Ou seja, a tokenização
  seria mais "padrão", do tipo palavra por palavra.

  ### Parâmetros:
  - texto: String contendo o texto a ser pré-processado/tokenizado.
  - caminho_para_salvar_tokenizacao: String contendo o caminho para salvar o
  arquivo do texto pré-processado/tokenizado.
  - silence: Bool que dirá para printar (True) ou não (False) informações ao decorrer
  da execução desta função.

  ### Retornos:
  - Bool que dirá se o processo de tokenização sem reconhecimento de entidades
  foi bem sucedido (True) ou não (False).
  """
  if texto:

    # O pré-processamento começa padronizando o texto
    texto = padronizacaoTexto(texto)

    # Inicialização da lista que conterá as listas de tokens (frases tokenizadas)
    lista_frases_tokenizadas = []

    # Obtenção das sentenças/frases do texto em questão usando NLTK
    sentencas = sent_tokenize(texto, language='portuguese')

    # Passando frase por frase
    for sentenca in sentencas:
      # Checando se a frase possue tokens suficientes para entrar pro nosso corpus pré-processado
      if checarQuantidadePalavrasNaFrase(frase=sentenca):
        # Inicialização da lista que conterá a frase tokenizada (tokens da frase em questão)
        lista_atual = []

        # Passando palavra por palavra usando o processamento de linguagem natural do spaCy para identificação de verbos
        for palavra in nlp_spacy(sentenca):
          try:
            if (not palavra.is_punct): # Se a palavra não for um tipo de pontuação
              if palavra.pos_ in ['VERB','AUX']: # Se a palavra for do tipo verbo ou auxiliar
                # Adicionaremos a palavra/token da forma lematizada, usando a ferramenta de lematização que importamos, sem ignorar formas ambíguas de lematização dos verbos
                lista_atual.append(lematizar(verb=palavra.orth_,ignore_ambiguous_verbs=False))
              else: # Se a palavra não for caracterizada como verbo pelo spaCy
                lista_atual.append(palavra.orth_) # Adicionaremos a forma textual/string da palavra analisada
          except Exception as e: # Caso ocorra algum erro inesperado trataremos da seguinte forma:
            if not silence: # Se o silêncio não for pedido, o executor do programa será notificado por meio da impressão da mensagem de erro na tela
              erro = f'{e.__class__.__name__}: {str(e)}'
              print(f'Erro ao processar texto na frase "{sentenca}":\n\t-->{erro}.\nProcesso de Tokenização com lematização total / Sem rec. de ents / spaCy')
            else: # Se o silêncio for pedido, simplesmente passaremos este bloco de código
              pass



        if lista_atual: # Se a lista de tokens atual não estiver vazia
          lista_frases_tokenizadas.append(lista_atual) # Adicionaremos a lista de tokens (frase tokenizada) atual a lista de frases tokenizadas

    if lista_frases_tokenizadas: # Se a lista de frases tokenizadas não estiver vazia
      lista_frases_tokenizadas = padronizacaoTokens(lista_frases_tokenizadas=lista_frases_tokenizadas) # Padronizaremos os tokens da lista de frases tokenizadas

      if lista_frases_tokenizadas: # Se, após a padronização de tokens, a lista continuar não estando vazia
        # Inicializamos o processo de salvar a lista de listas de tokens (frases tokenizadas)
        status_save, msg_save = salvarArquivoMsgPack(nome_variavel='pre_processamento_s_re',
                                                     variavel_em_questao=lista_frases_tokenizadas,
                                                     pasta_para_salvar=caminho_para_salvar_tokenizacao)
        if not status_save: # Caso ocorra algum problema na etapa de salvar o arquivo pré-processado, será printado a mensagem de erro
          print('\n!',msg_save,'\n\n')
          return False
        else:
          return True
  return False

def preProcessarTextoComRE(texto : str,
                           caminho_para_salvar_tokenizacao : str,
                           silence : bool = True) -> bool:
  """
  Função responsável por pré-processar o texto com a tokenização usando o
  reconhecimento de entidades (tokenizar entidades nomeadas). Ou seja, a tokenização
  seria mais elaborada, podendo tokenizar mais de uma palavra num mesmo token
  (usando "_" no lugar do espaço em branco).

  ### Parâmetros:
  - texto: String contendo o texto a ser pré-processado/tokenizado.
  - caminho_para_salvar_tokenizacao: String contendo o caminho para salvar o
  arquivo do texto pré-processado/tokenizado.
  - silence: Bool que dirá para printar (True) ou não (False) informações ao decorrer
  da execução desta função.

  ### Retornos:
  - Bool que dirá se o processo de tokenização com reconhecimento de entidades
  foi bem sucedido (True) ou não (False).
  """
  if texto:

    # O pré-processamento começa padronizando o texto
    texto = padronizacaoTexto(texto)

    # Inicialização da lista que conterá as entidades reconhecidas no texto
    lista_entidades = []
    # Inicialização da lista que conterá as listas de tokens (frases tokenizadas)
    lista_frases_tokenizadas = []

    # Obtenção das sentenças/frases do texto em questão usando NLTK
    sentencas = sent_tokenize(texto, language='portuguese')

    # Passando frase por frase
    for sentenca in sentencas:
      # Checando se a frase possue tokens suficientes para entrar pro nosso corpus pré-processado
      if checarQuantidadePalavrasNaFrase(frase=sentenca):
        # Processamento do texto via spaCy para obtenção das entidades reconhecidas no texto em questão
        doc = nlp_spacy(sentenca)

        # Lista de tuplas onde cada tupla possue 2 elementos sendo o primeiro a entidade reconhecida pelo spaCy no texto e o
        # segundo a mesma entidade formatada (caracteres minúsculos, remoção de caractéres especiais) e substituição do espaço em branco pelo "_"
        lista_tuplas_entidades = [(ent.orth_,transformarTextoSubstituindoCaracteres(texto=formatarTexto(texto=ent.orth_,
                                                                                                        remover_caracteres_estranhos=False,
                                                                                                        remover_espacos_em_branco_em_excesso=False,),
                                                                                    caracteres=' ',
                                                                                    substituir_por='_')) for ent in doc.ents]

        # Lista de tuplas onde cada tupla possue 2 elementos sendo o primeiro a entidade nomeada adicionada manualmente e encontrada na frase e o
        # segundo a mesma entidade formatada (caracteres minúsculos, remoção de caractéres especiais) e substituição do espaço em branco pelo "_"
        lista_tuplas_entidades_manuais = [(entidade_normal, transformarTextoSubstituindoCaracteres(texto=formatarTexto(texto=entidade_normal,
                                                                                                          remover_caracteres_estranhos=False,
                                                                                                          remover_espacos_em_branco_em_excesso=False),
                                                                                                    caracteres=' ',
                                                                                                    substituir_por='_')) for entidade_normal in lista_entidades_nomeadas_manuais_ordenada if entidade_normal in sentenca]

        # Inicialização da lista que conterá a frase tokenizada (tokens da frase em questão)
        lista_atual = []

        # Substituição das entidades reconhecidas pelo spaCy pelas entidades formatadas na frase
        for entidade_normal,entidade_formatada in lista_tuplas_entidades:
          sentenca = sentenca.replace(entidade_normal,entidade_formatada)
          lista_entidades.append(entidade_formatada)

        # Substituição das entidades nomeadas "manuais" formatadas na frase que as possuem
        for entidade_normal, entidade_formatada in lista_tuplas_entidades_manuais:
          sentenca = substituirEntidadeNomeadaManual(texto=sentenca,entidade_normal=entidade_normal,entidade_formatada=entidade_formatada)
          lista_entidades.append(entidade_formatada)


        # Passando palavra por palavra usando o processamento de linguagem natural do spaCy para identificação de verbos
        # Aqui acontece o mesmo processo descrito na função Sem o Reconhecimento de Entidades
        for palavra in nlp_spacy(sentenca):
          try:
            if (not palavra.is_punct):
              if palavra.orth_ not in lista_entidades:
                if palavra.pos_ in ['VERB','AUX']:
                  lista_atual.append(lematizar(verb=palavra.orth_,ignore_ambiguous_verbs=False))
                else:
                  lista_atual.append(palavra.orth_)
              else:
                lista_atual.append(palavra.orth_)
          except Exception as e:
            if not silence:
              erro = f'{e.__class__.__name__}: {str(e)}'
              print(f'Erro ao processar texto na frase "{sentenca}":\n\t-->{erro}.\nProcesso de Tokenização com lematização total / Sem rec. de ents / spaCy')
            else:
              pass

        if lista_atual:
          lista_frases_tokenizadas.append(lista_atual)

    if lista_frases_tokenizadas:
      lista_tokens_personalizados = [entidade for entidade in lista_entidades] # Criando uma lista de tokens personalizados (que não serão formatados) com as entidades reconhecidas no texto
      lista_frases_tokenizadas = padronizacaoTokens(lista_frases_tokenizadas=lista_frases_tokenizadas,lista_tokens_personalizados=lista_tokens_personalizados)

      if lista_frases_tokenizadas:
        status_save, msg_save = salvarArquivoMsgPack(nome_variavel='pre_processamento_c_re',
                                                     variavel_em_questao=lista_frases_tokenizadas,
                                                     pasta_para_salvar=caminho_para_salvar_tokenizacao)
        if not status_save:
          print('\n!',msg_save,'\n\n')
          return False
        else:
          return True
  else:
    print('\n!Texto inválido (vazio ao coletar).\n')

  return False

def listagemDeTrabalhos(lista_de_trabalhos : list,
                        n_programa : int,
                        numero_max_programas : int = 23) -> list:
  """
  Função responsável por separar os caminhos de todos os trabalhos pela quantidade
  de programas executores nesta etapa de pré-processamento, organizando as quais
  trabalhos serão contemplados por qual programa, relacionando o número do programa,
  a quantidade de trabalhos total e o número máximo de programas que serão executados
  para pré-processar o corpus de textos.

  ### Parâmetros:
  - lista_de_trabalhos: Lista contendo elementos do tipo strings referentes ao
  caminho dos trabalhos.
  - n_programa: Inteiro para dizer que programa está sendo executado neste notebook
  (cada programa deve ter a variável n_programa setada para o número da execução).
  - numero_max_programas: Inteiro para dizer em quantos programas o pré-processamento
  será dividido. No caso foram utilizados 7 contas da Google para executar este
  pré-processamento. Uma conta com assinatura do Colab podendo rodar 5 programas
  e outras 6 contas gratuitas podendo rodar, cada uma, 3 programas em simultâneo.

  ### Retornos:
  - Lista contendo os caminhos dos trabalhos que serão contemplados pelo programa
  em questão.
  """
  dic_programas_trabalhos = {}
  for np in range(1,numero_max_programas+1):
    dic_programas_trabalhos[f'Prog {np}'] = []

  for i in range(len(lista_de_trabalhos)):
    if (i+1) <= numero_max_programas:
      num = i + 1
      dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
    else:
      if (i+1)%numero_max_programas != 0:
        num = (i+1)%numero_max_programas
        dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
      else:
        num = numero_max_programas
        dic_programas_trabalhos[f'Prog {num}'].append(lista_de_trabalhos[i])
  return dic_programas_trabalhos[f'Prog {n_programa}']

def verificarDataArquivo(caminho_arquivo : str,
                         data_limite : datetime.datetime) -> bool:
  """
  Função responsável por verificar se o arquivo analisado foi criado/alterado antes
  ou depois de uma data limite.

  ### Parâmetros:
  - caminho_arquivo: String contendo o caminho para o arquivo a ser analisado a
  data.
  - data_limite: Datetime referente a data estipulada como limitante para tomada
  de alguma decisão.

  ### Retornos:
  - Bool que dirá se o arquivo foi criado/alterado depois da data limite (True)
  ou se foi criado/alterado antes da data limite (False).
  """
  try:
    data_hora_criacao_arquivo = datetime.datetime.fromtimestamp(os.path.getctime(caminho_arquivo))
    # Verificando se data_hora_criacao_arquivo foi antes de data_limite
    if data_hora_criacao_arquivo > data_limite:
      # Foi criado/alterado depois
      return True
    else:
      # Foi criado/alterado antes
      return False
  except Exception as e:
    # Arquivo não foi criado ainda
    return False

def preProcessarTexto(txt_total : str,
                      caminho_pasta_salvar_pre_processamento : str,
                      com_re : bool = True,
                      sem_re : bool = True,
                      substituir_arquivos_antigos : bool = True,
                      verificar_data_limite : bool = True,
                      data_limite : datetime.datetime = datetime.datetime.now(),
                      silence : bool = True) -> tuple[str,float]:
  """
  Função responsável por organizar o pré-processamento de um determinado texto,
  chamando as outras funções que realizam este processo.

  ### Parâmetros:
  - txt_total: String contendo o texto a ser pré-processado.
  - caminho_pasta_salvar_pre_processamento: String contendo o caminho até a pasta
  que armazenará o arquivo do texto pré-processado.
  - com_re: Bool que dirá se será executado o pré-processamento com Reconhecimento
  de Entidades no texto (True) ou não (False).
  - sem_re: Bool que dirá se será executado o pré-processamento sem Reconhecimento
  de Entidades no texto (True) ou não (False).
  - substituir_arquivos_antigos: Bool que dirá se, caso exista arquivo pré-processado
  para o texto em questão, será substituído (True) ou não (False).
  - verificar_data_limite: Bool que dirá se será (True) ou não (False) feita a
  análise da data de criação/alteração do arquivo de pré-processamento do texto
  atual e cheque a data_limite para saber se este foi criado/alterado antes
  ou depois dela.
  - data_limite: Datetime referente a data estipulada como limitante para a substituição
  ou não (a depender da análise) dos arquivos de pré-processamento, caso seja optado
  por isto.
  - silence: Bool que dirá se será mostrado/printado no console (True) ou não (False)
  as informações ao decorrer da execução do pré-processamento.

  ### Retornos:
  - Tupla de dois elementos sendo o primeiro uma string para ser exibida no console
  (referente a informação do pré-processamento do texto atual) e o segundo um float
  referente ao tempo de duração do pré-processamento do texto atual.
  """
  string_console = ''
  t1 = time.time()

  if sem_re:
    if verificar_data_limite:
      if not verificarDataArquivo(caminho_arquivo=os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_s_re.msgpack'),data_limite=data_limite):
        if not silence:
          print('Foi encontrado arquivo antigo de pré-processamento SRE...')
        if (os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_s_re.msgpack')) and substituir_arquivos_antigos) or (not os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_s_re.msgpack'))):
          if preProcessarTextoSemRE(texto=txt_total,caminho_para_salvar_tokenizacao=caminho_pasta_salvar_pre_processamento):
            string_console += '\nTokenização S RE: OK\n'
          else:
            string_console += '\nTokenização S RE: FALHOU\n'
            if not silence:
              print('\n! Problema ao salvar arquivo de pre_processamento sem RE\n')
      else:
        if not silence:
          print('Foi encontrado arquivo recente de pré-processamento...')
    else:
      if (os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_s_re.msgpack')) and substituir_arquivos_antigos) or (not os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_s_re.msgpack'))):
        if preProcessarTextoSemRE(texto=txt_total,caminho_para_salvar_tokenizacao=caminho_pasta_salvar_pre_processamento):
          string_console += '\nTokenização S RE: OK\n'
        else:
          string_console += '\nTokenização S RE: FALHOU\n'
          if not silence:
            print('\n! Problema ao salvar arquivo de pre_processamento sem RE\n')

  if com_re:
    if verificar_data_limite:
      if not verificarDataArquivo(caminho_arquivo=os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_c_re.msgpack'),data_limite=data_limite):
        if not silence:
          print('Foi encontrado arquivo antigo de pré-processamento CRE...')
        if (os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_c_re.msgpack')) and substituir_arquivos_antigos) or (not os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_c_re.msgpack'))):
          if preProcessarTextoComRE(texto=txt_total,caminho_para_salvar_tokenizacao=caminho_pasta_salvar_pre_processamento,silence=silence):
            string_console += '\nTokenização C RE: OK\n'
          else:
            string_console += '\nTokenização C RE: FALHOU\n'
            if not silence:
              print('\n! Problema ao salvar arquivo de pre_processamento com RE\n')
      else:
        if not silence:
          print('Foi encontrado arquivo recente de pré-processamento...')
    else:
      if (os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_c_re.msgpack')) and substituir_arquivos_antigos) or (not os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_c_re.msgpack'))):
        if preProcessarTextoComRE(texto=txt_total,caminho_para_salvar_tokenizacao=caminho_pasta_salvar_pre_processamento,silence=silence):
          string_console += '\nTokenização C RE: OK\n'
        else:
          string_console += '\nTokenização C RE: FALHOU\n'
          if not silence:
            print('\n! Problema ao salvar arquivo de pre_processamento com RE\n')
  t2 = time.time()
  tempo_exec_pps = round((t2-t1),4)
  return string_console, tempo_exec_pps


### main

In [ ]:
def main(n_programa : int):
  """
  Função responsável por dar início a execução da etapa de pré-processamento e que
  se encarregará da chamada das outras funções para executar, de fato, o pré-processamento
  do corpus de textos.

  ### Parâmetros:
  - n_programa: Inteiro contendo a numeração do programa que está executando este
  notebook.

  ### Retornos:
  - None: nenhum retorno está previsto para esta função.
  """
  caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

  # caminho_colecoes_word_embeddings = f'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_tokenizados/Colecoes_tokenizadas_{n_programa}'

  pasta_central = f'/content/drive/MyDrive/WOKE PP PROGRAMA {n_programa}'



  if not os.path.exists(pasta_central):
    print(f'\n\t! 1. Crie uma pasta denominada de "{os.path.basename(pasta_central)}" no seu Drive (na pasta principal).')
    print('\n\n\t! 2. Dê permissão para as contas de e-mail do Grupo de Estudos e do Igor poderem editá-la.')
    print('\n\n\t! 3. Desconecte e conecte novamente este ambiente de execução e execute-o novamente.')
    print('\n\n\n\t--> EM CASO DE DÚVIDAS ENTRE NO LINK ABAIXO PARA ASSISTIR O VÍDEO EXPLICATIVO:')
    print('\n\t--> Link:  https://drive.google.com/file/d/1RioMR1kGrYHYG5-GTVZVx5mZFElSN7Sm/view?usp=sharing \n')
  else:

    caminho_colecoes_word_embeddings = os.path.join(pasta_central,f'Colecoes_tokenizadas_{n_programa}')

    if not os.path.exists(caminho_colecoes_word_embeddings):
      os.makedirs(caminho_colecoes_word_embeddings)

    lista_de_colecoes_extracao = [os.path.join(caminho_colecoes_extracao_textos,colecao) for colecao in os.listdir(caminho_colecoes_extracao_textos) if '.' not in colecao]

    lista_nomes_colecoes_extracao = [os.path.basename(colecao) for colecao in lista_de_colecoes_extracao]

    dic_colecoes_extracao = {chave: valor for chave, valor in zip(lista_nomes_colecoes_extracao, lista_de_colecoes_extracao)}

    tempos_exec_pps = []


    if os.path.exists(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/lista_caminho_arquivos.msgpack'):
      lista_caminho_arquivos = abrirArquivoMsgPack(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/lista_caminho_arquivos.msgpack')
    else:
      print('\n\n\tNão encontrei o arquivo da lista de arquivos :(\n\tMontando lista de arquivos...\n\n\tIsso pode levar alguns minutos...\n\n')
      lista_caminho_arquivos = []

      # Criação da lista de dicionários que conterão todos os caminhos para o posterior pré-processamento:
      for contagem,nome_colecao in enumerate(sorted(dic_colecoes_extracao.keys())):
        colecao = dic_colecoes_extracao[nome_colecao]
        lista_de_anos_colecao_extracao = sorted([os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()])
        for ano in lista_de_anos_colecao_extracao:
          lista_trabalhos_ano_colecao_extracao = sorted([os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')])
          for trabalho in lista_trabalhos_ano_colecao_extracao:
            lista_caminho_arquivos.append(trabalho)


    output.clear()
    print('\n\n\tSelecionando os arquivos para este programa...\n\n')
    # Gerando a lista única de dicionários de caminhos para este programa em específico
    lista_caminho_arquivos_atual = listagemDeTrabalhos(lista_de_trabalhos=lista_caminho_arquivos,n_programa=n_programa)

    caminho_lista_caminhos_trabalhos_ja_pre_processados = os.path.join(pasta_central,f'lista_caminhos_trabalhos_ja_pre_processados_{n_programa}.msgpack')
    if os.path.exists(caminho_lista_caminhos_trabalhos_ja_pre_processados):
      lista_caminhos_trabalhos_ja_pre_processados = abrirArquivoMsgPack(caminho_lista_caminhos_trabalhos_ja_pre_processados)
    else:
      lista_caminhos_trabalhos_ja_pre_processados = []

    qtd_trabalhos = len(lista_caminho_arquivos_atual)

    output.clear()
    print('\n\n\tIniciando...\n\n')
    # Inicializando o arquivo de texto gerado por este programa:
    lista_de_colecoes_que_serao_contempladas = sorted(set([os.path.basename(os.path.dirname(os.path.dirname(caminho_trabalho_atual))) for caminho_trabalho_atual in lista_caminho_arquivos_atual]))
    string_console = f'Programa {n_programa}\n\nColeções contempladas:\n'
    for colecao in lista_de_colecoes_que_serao_contempladas:
        string_console += f'- {colecao}\n'
    string_console += '\n'
    caminho_arquivo_console_programa = os.path.join(pasta_central,f'Console - Programa {n_programa}.txt')
    if not os.path.exists(caminho_arquivo_console_programa):
      with open(caminho_arquivo_console_programa,'w',encoding='utf-8') as f:
          f.write(string_console)

    for contagem,caminho_trabalho_atual in enumerate(lista_caminho_arquivos_atual):
      trabalho = os.path.basename(caminho_trabalho_atual)
      ano = os.path.basename(os.path.dirname(caminho_trabalho_atual))
      colecao = os.path.basename(os.path.dirname(os.path.dirname(caminho_trabalho_atual)))
      trab_atual = [colecao,ano,trabalho]

      if trab_atual not in lista_caminhos_trabalhos_ja_pre_processados:

        porcentagem_realizada = str(round(((contagem+1)*100/qtd_trabalhos),2)).replace('.',',')+'%'

        if tempos_exec_pps:
          media_pp = str(round(mean(tempos_exec_pps),2))
          string_console = '\n'+'-'*100+'\n'+f'Coleção: {colecao} - {contagem+1} de {qtd_trabalhos} ({porcentagem_realizada}) - Tempo médio de exec-pps: {media_pp} segundos.\n'+'-'*100+'\n'
        else:
          string_console = '\n'+'-'*100+'\n'+f'Coleção: {colecao} - {contagem+1} de {qtd_trabalhos} ({porcentagem_realizada})\n'+'-'*100+'\n'

        string_console += f"{ano}\n{trabalho}\n"
        lista_arquivos_trabalho_ano_colecao_extracao = [os.path.join(caminho_trabalho_atual,arquivo) for arquivo in os.listdir(caminho_trabalho_atual) if arquivo in ['notas_de_rodape.txt','texto_principal.txt']]
        txt_total = ''
        for arquivo in lista_arquivos_trabalho_ano_colecao_extracao:
          try:
            txt_total += coletarTextoDeArquivoTxt(caminho_arquivo=arquivo,tipo_de_encoding='utf-8')+'.  '
          except Exception as e:
            print(f'{e.__class__.__name__}: {str(e)}')
            pass
        txt_total = txt_total.strip()
        if txt_total != '':
          caminho_pasta_salvar_pre_processamento = os.path.join(caminho_colecoes_word_embeddings,colecao,ano,trabalho)
          if not os.path.exists(caminho_pasta_salvar_pre_processamento):
            os.makedirs(caminho_pasta_salvar_pre_processamento)

          add_string_console, tempo_exec_pps= preProcessarTexto(txt_total=txt_total,caminho_pasta_salvar_pre_processamento=caminho_pasta_salvar_pre_processamento,
                                                                com_re=True,sem_re=False,substituir_arquivos_antigos=False,
                                                                verificar_data_limite=False,data_limite=datetime.datetime(2024, 4, 24), # Setando a data limite para arquivo mais antigos (mais de 2 dias atrás, pois o pré-processamento foi quebrado no meio ontem)
                                                                silence=False)
          string_console += add_string_console
          if tempo_exec_pps > 2:
            tempos_exec_pps.append(tempo_exec_pps)
            tempo_pp = round(tempo_exec_pps,2)
            string_console += '\t--> Tempo para executar estes pre-processamentos: '+str(tempo_pp)+'\n'

        atualizarConsole(string_console)
        # time.sleep(2)
        if 'Tempo para executar estes pre-processamentos' in string_console:
          with open(caminho_arquivo_console_programa,'a',encoding='utf-8') as f:
            f.write(string_console)
        lista_caminhos_trabalhos_ja_pre_processados.append([colecao,ano,trabalho])
        salvarArquivoMsgPack(nome_variavel=f'lista_caminhos_trabalhos_ja_pre_processados_{n_programa}',variavel_em_questao=lista_caminhos_trabalhos_ja_pre_processados,pasta_para_salvar=os.path.dirname(caminho_lista_caminhos_trabalhos_ja_pre_processados))

# Execução do Pré-processamento

In [ ]:
n_programa = 1

main(n_programa)


----------------------------------------------------------------------------------------------------
Coleção: Urbanismo_Historia_e_Arquitetura_da_Cidade - 1334 de 1334 (100,0%) - Tempo médio de exec-pps: 38.72 segundos.
----------------------------------------------------------------------------------------------------
2016
Trabalho 65

Tokenização C RE: OK
	--> Tempo para executar estes pre-processamentos: 17.51



# Copiar arquivos para a pasta principal

Após realizado o pré-processamento de todos os programas, compartilhando as pastas das contas externas com esta conta do Grupo de Estudos (a qual irá armazenar todos os arquivos de pré-processamento numa pasta só), é necessário criar um atalho destas pastas ("WOKE PP PROGRAMA X", sendo X o número do programa) para a área do Meu Drive.

Depois, por meio das execuções abaixo, copiar os arquivos pré-processados pelas outras contas para uma pasta geral, onde esta conta (do Grupo de Estudos) será proprietária de todos os arquivos (assim as outras contas podem excluir os arquivos e esta conta segue sendo a portadora de todos os dados).

In [ ]:
import shutil

def copiarArquivo(caminho_arquivo_original : str,
                  pasta_destino : str):
  try:
    if not os.path.exists(pasta_destino):
      os.makedirs(pasta_destino)

    shutil.copy(caminho_arquivo_original, pasta_destino)
  except Exception as e:
    print(f'\n! Falha: {e.__class__.__name__}: {str(e)}')
    return False
  else:
    return True

In [ ]:
import os

pasta_para_alocacao = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

lista_pastas_tokenizacao_outros_programas = [r'/content/drive/MyDrive/WOKE PP PROGRAMA 1',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 2',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 3',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 4',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 5',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 6',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 7',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 8',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 9',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 10',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 11',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 12',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 13',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 14',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 15',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 16',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 17',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 18',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 19',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 20',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 21',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 22',
                                             r'/content/drive/MyDrive/WOKE PP PROGRAMA 23']
qtd_pastas = len(lista_pastas_tokenizacao_outros_programas)

for pasta in lista_pastas_tokenizacao_outros_programas:
  np = pasta[-2:].strip()
  nome_pasta_colecoes = f'Colecoes_tokenizadas_{np}'
  caminho_pasta_colecoes_tokenizadas = os.path.join(pasta,nome_pasta_colecoes)
  lista_colecoes = [os.path.join(caminho_pasta_colecoes_tokenizadas,colecao) for colecao in os.listdir(caminho_pasta_colecoes_tokenizadas) if '.' not in colecao]
  qtd_colecoes = len(lista_colecoes)
  lista_colecoes_atual = listagemDeTrabalhos(lista_de_trabalhos = lista_colecoes,n_programa=1,numero_max_programas=5)
  qtd_colecoes_atual = len(lista_colecoes_atual)

  for cont,colecao in enumerate(lista_colecoes_atual):
    output.clear()
    print('Pasta',os.path.basename(pasta),'de',qtd_pastas)
    print('Coleção',cont+1,'de',qtd_colecoes_atual)
    lista_anos = [os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()]
    for ano in lista_anos:
      lista_trabalhos = [os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')]
      for trabalho in lista_trabalhos:
        lista_arquivos = [os.path.join(trabalho,arquivo) for arquivo in os.listdir(trabalho) if arquivo in ['pre_processamento_c_re.msgpack','pre_processamento_s_re.msgpack']]

        nome_trabalho = os.path.basename(trabalho)
        nome_ano = os.path.basename(os.path.dirname(trabalho))
        nome_colecao = os.path.basename(os.path.dirname(os.path.dirname(trabalho)))

        caminho_pasta_trabalho_para_alocao = os.path.join(pasta_para_alocacao,nome_colecao,nome_ano,nome_trabalho)

        if not os.path.exists(os.path.join(caminho_pasta_trabalho_para_alocao,'pre_processamento_c_re.msgpack')): # Certificando que não vamos inserir arquivos com duplicidade (caso ocorra)
          if not os.path.exists(caminho_pasta_trabalho_para_alocao):
            os.makedirs(caminho_pasta_trabalho_para_alocao)
          for arquivo in lista_arquivos:
            if not copiarArquivo(caminho_arquivo_original=arquivo,pasta_destino=caminho_pasta_trabalho_para_alocao):
              print(nome_colecao,'-',nome_ano,'-',nome_trabalho,'-->','PROBLEMA')

Pasta WOKE PP PROGRAMA 23 de 23
Coleção 20 de 20


# Validação dos processos (executar apenas quando todos os programas encerrarem)

Esta etapa não está necessariamente ligada as execuções de pré-processamento (todo processo já foi descrito à cima), mas como faz menção a elas, decidi deixar neste notebook mesmo.

Esta execução é para realizar uma validação do pré-processamento (ao encerrar as execuções de todos os outros programas que o estiverem fazendo). Esta validação é feita comparando a quantidade de arquivos de texto extraído e a quantidade de arquivos de pré-processamento por coleção. Caso algum arquivo de pré-processamento não exista será notificado no console com a mensagem de que não foi encontrado arquivo pré-processado para o determinado trabalho que possui seu texto extraído.

In [4]:
pasta_central = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

qtd_txt_pp = 0

lista_colecoes = sorted([os.path.join(pasta_central,c) for c in os.listdir(pasta_central) if '.' not in c])
qtd_col = len(lista_colecoes)
for cont,colecao in enumerate(lista_colecoes):
  for ano in [os.path.join(colecao,a) for a in os.listdir(colecao) if a.isdigit()]:
    for trabalho in [os.path.join(ano,t) for t in os.listdir(ano) if t.startswith('Trabalho')]:
      if os.path.exists(os.path.join(trabalho,'pre_processamento_c_re.msgpack')):
        qtd_txt_pp += 1
  output.clear()
  print(os.path.basename(colecao),cont+1,'de',qtd_col)
  print(qtd_txt_pp)

# output.clear()
print('\n\nTotal de trabalhos pré-processados:')
print(qtd_txt_pp)

Direito_Mestrado_Profissional 28 de 106
7388


KeyboardInterrupt: 

In [ ]:
caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

caminho_colecoes_word_embeddings = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

lista_de_colecoes_extracao = [os.path.join(caminho_colecoes_extracao_textos,colecao) for colecao in os.listdir(caminho_colecoes_extracao_textos) if '.' not in colecao] #  and colecao in ['Filosofia','Geografia','Geologia','Historia','Psicologia','Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas','Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas','Servico_Social','Sociologia_e_Ciencia_Politica','Sociologia_Politica','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Ensino_de_Historia_Mestrado_Profissional'] # pro caso do CFH

print('Quantidade de coleções extração:',len(lista_de_colecoes_extracao))
print('Quantidade de coleções word embeddings:',len([colecao for colecao in os.listdir(caminho_colecoes_word_embeddings) if '.' not in colecao]))

lista_arquivos_sem_pp = []
num_arq_sem_pp = 0

qtd_max_colecoes = len(lista_de_colecoes_extracao)

for cont,colecao in enumerate(sorted(lista_de_colecoes_extracao)):

  print('\n'+'-'*100+'\n'+f'Coleção: {os.path.basename(colecao)} - {cont+1} de {qtd_max_colecoes}\n'+'-'*100+'\n')

  lista_de_anos_colecao_extracao = sorted([os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()])

  qtd_colecao_txt = 0
  qtd_colecao_pp = 0

  for ano in lista_de_anos_colecao_extracao:

    lista_trabalhos_ano_colecao_extracao = sorted([os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')])

    for trabalho in lista_trabalhos_ano_colecao_extracao:
      if os.path.exists(os.path.join(trabalho,'texto_principal.txt')):
        qtd_colecao_txt += 1
        if not os.path.exists(os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho),'pre_processamento_c_re.msgpack')):
          lista_arquivos_sem_pp.append((os.path.join(trabalho,'texto_principal.txt'),os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho))))
        else:
          qtd_colecao_pp += 1
  if qtd_colecao_txt != qtd_colecao_pp:
    print('\n!!!',os.path.basename(colecao),'qtd_txt',qtd_colecao_txt,'qtd_pp',qtd_colecao_pp,'\n')
    num_arq_sem_pp += qtd_colecao_txt - qtd_colecao_pp
  else:
    print('\nOK\n')

  print('\nNúmero de arquivos sem pré-processamento:',num_arq_sem_pp,'\n')

Quantidade de coleções extração: 112
Quantidade de coleções word embeddings: 106

----------------------------------------------------------------------------------------------------
Coleção: Administracao - 1 de 112
----------------------------------------------------------------------------------------------------


OK


Número de arquivos sem pré-processamento: 0 


----------------------------------------------------------------------------------------------------
Coleção: Administracao_Universitaria_Mestrado_Profissional - 2 de 112
----------------------------------------------------------------------------------------------------


OK


Número de arquivos sem pré-processamento: 0 


----------------------------------------------------------------------------------------------------
Coleção: Agroecossistemas - 3 de 112
----------------------------------------------------------------------------------------------------


OK


Número de arquivos sem pré-processamento: 0 


---------

Foi notado que ocorreu algum tipo de imprevisto no pré-processamento, pois existiam 1085 trabalhos nas pastas de textos extraídos que estavam sem os seus arquivos correspondentes de pré-processamento. Depois foi analisado com cautela e foi visualizado que, por executar mais de um programa em simultâneo algumas criações de pastas ficaram errôneas, pois foram criadas praticamente ao mesmo tempo em programas diferentes, criando algumas cópias de pastas de coleções para uma mesma coleção (por exemplo tinha a pasta da coleção "Biologia", mas também tinha uma pasta da coleção "Biologia (1)", que seria a cópia) e de alguns anos de algumas coleções também. Tendo isso em vista foi planejado mais uma leve de execuções para corrigir este problema e pré-processar os textos que não tinham sido contemplados no pré-processamento realizado.

O resultado está salvo nas janelas de outputs das células de execução abaixo e, já adiantando, ocorreu perda de apenas 6 textos extraídos no pré-processamento (por motivos desconhecidos).

### Ajeitar arquivos sem pré-processamento

In [ ]:
lista_arquivos_sem_pp[0]

('/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções/Ciencia_e_Engenharia_de_Materiais/2010/Trabalho 319/texto_principal.txt',
 '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados/Ciencia_e_Engenharia_de_Materiais/2010/Trabalho 319')

In [ ]:
len(lista_arquivos_sem_pp)

1085

In [ ]:
lista_nova = []

for tupla1,tupla2 in lista_arquivos_sem_pp:
  lista_nova.append([tupla1,tupla2])

print(len(lista_nova))
print(lista_nova[0])

1085
['/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções/Ciencia_e_Engenharia_de_Materiais/2010/Trabalho 319/texto_principal.txt', '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados/Ciencia_e_Engenharia_de_Materiais/2010/Trabalho 319']


In [ ]:
!pip install msgpack==1.0.7
import msgpack
salvarArquivoMsgPack(nome_variavel='lista_arquivos_sem_pp',variavel_em_questao=lista_nova,pasta_para_salvar=r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.8
    Uninstalling msgpack-1.0.8:
      Successfully uninstalled msgpack-1.0.8


(True,
 'Variável lista_arquivos_sem_pp.msgpack salva com sucesso no formato .msgpack')

In [ ]:
lista_salva_arquivos_sem_pp = abrirArquivoMsgPack(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados/lista_arquivos_sem_pp.msgpack')

In [ ]:
print(len(lista_salva_arquivos_sem_pp))
lista_salva_arquivos_sem_pp[0]

1085


['/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções/Ciencia_e_Engenharia_de_Materiais/2010/Trabalho 319/texto_principal.txt',
 '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados/Ciencia_e_Engenharia_de_Materiais/2010/Trabalho 319']

In [ ]:
lista_arquivos_sem_pp_atual = listagemDeTrabalhos(lista_de_trabalhos=lista_salva_arquivos_sem_pp,n_programa=1,numero_max_programas=5)
qtd_total = len(lista_arquivos_sem_pp_atual)
cont = 0
for caminho_arquivo_txt,caminho_pasta_salvar_pre_processamento in lista_arquivos_sem_pp_atual:
  cont+=1
  print('\n'+'-'*100+'\n'+f'Coleção: {cont} de {qtd_total}\n'+'-'*100+'\n')
  txt_total = ''
  pasta_trabalho_extracao = os.path.dirname(caminho_arquivo_txt)
  # print(pasta_trabalho_extracao)
  # break
  for arquivo in [os.path.join(pasta_trabalho_extracao,arq) for arq in os.listdir(pasta_trabalho_extracao) if arq in ['texto_principal.txt','notas_de_rodape.txt']]:
    try:
      txt_total += coletarTextoDeArquivoTxt(caminho_arquivo=arquivo,tipo_de_encoding='utf-8')+'.  '
    except Exception as e:
      print(f'{e.__class__.__name__}: {str(e)}')
      pass
  txt_total = txt_total.strip()
  if txt_total != '':
    if not os.path.exists(caminho_pasta_salvar_pre_processamento):
      os.makedirs(caminho_pasta_salvar_pre_processamento)
    if not os.path.exists(os.path.join(caminho_pasta_salvar_pre_processamento,'pre_processamento_c_re.msgpack')):
      add_string_console, tempo_exec_pps = preProcessarTexto(txt_total=txt_total,caminho_pasta_salvar_pre_processamento=caminho_pasta_salvar_pre_processamento,
                                                            com_re=True,sem_re=False,substituir_arquivos_antigos=False,
                                                            verificar_data_limite=False,data_limite=datetime.datetime(2024, 4, 24), # Setando a data limite para arquivo mais antigos (mais de 2 dias atrás, pois o pré-processamento foi quebrado no meio ontem)
                                                            silence=False)
    else:
      print('\nArquivo de pré-processamento já criado e salvo.\n')


----------------------------------------------------------------------------------------------------
Coleção: 1 de 217
----------------------------------------------------------------------------------------------------


! Problema ao salvar arquivo de pre_processamento com RE


----------------------------------------------------------------------------------------------------
Coleção: 2 de 217
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Coleção: 3 de 217
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Coleção: 4 de 217
----------------------------------------------------------------------------------------------------


----------------------------------------------------

In [ ]:
caminho_colecoes_extracao_textos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via API DSpace Protocolo OAI-PMH/Resultados/Coleções'

caminho_colecoes_word_embeddings = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

lista_de_colecoes_extracao = [os.path.join(caminho_colecoes_extracao_textos,colecao) for colecao in os.listdir(caminho_colecoes_extracao_textos) if ('.' not in colecao)] #  and colecao in ['Filosofia','Geografia','Geologia','Historia','Psicologia','Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas','Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas','Servico_Social','Sociologia_e_Ciencia_Politica','Sociologia_Politica','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Ensino_de_Historia_Mestrado_Profissional']

print('Quantidade de coleções extração:',len(lista_de_colecoes_extracao))
print('Quantidade de coleções word embeddings:',len([colecao for colecao in os.listdir(caminho_colecoes_word_embeddings) if '.' not in colecao]))

lista_arquivos_sem_pp = []
num_arq_sem_pp = 0

qtd_max_colecoes = len(lista_de_colecoes_extracao)

for cont,colecao in enumerate(sorted(lista_de_colecoes_extracao)):

  print('\n'+'-'*100+'\n'+f'Coleção: {os.path.basename(colecao)} - {cont+1} de {qtd_max_colecoes}\n'+'-'*100+'\n')

  lista_de_anos_colecao_extracao = sorted([os.path.join(colecao,ano) for ano in os.listdir(colecao) if ano.isdigit()])

  qtd_colecao_txt = 0
  qtd_colecao_pp = 0

  for ano in lista_de_anos_colecao_extracao:

    lista_trabalhos_ano_colecao_extracao = sorted([os.path.join(ano,trabalho) for trabalho in os.listdir(ano) if trabalho.startswith('Trabalho')])

    for trabalho in lista_trabalhos_ano_colecao_extracao:
      if os.path.exists(os.path.join(trabalho,'texto_principal.txt')):
        qtd_colecao_txt += 1
        if not os.path.exists(os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho),'pre_processamento_c_re.msgpack')):
          lista_arquivos_sem_pp.append((os.path.join(trabalho,'texto_principal.txt'),os.path.join(caminho_colecoes_word_embeddings,os.path.basename(colecao),os.path.basename(ano),os.path.basename(trabalho))))
        else:
          qtd_colecao_pp += 1
  if qtd_colecao_txt != qtd_colecao_pp:
    print('\n!!!',os.path.basename(colecao),'qtd_txt',qtd_colecao_txt,'qtd_pp',qtd_colecao_pp,'\n')
    num_arq_sem_pp += qtd_colecao_txt - qtd_colecao_pp
  else:
    print('\nOK\n')

  print('\nNúmero de arquivos sem pré-processamento:',num_arq_sem_pp,'\n')

Quantidade de coleções extração: 112
Quantidade de coleções word embeddings: 106

----------------------------------------------------------------------------------------------------
Coleção: Administracao - 1 de 112
----------------------------------------------------------------------------------------------------


OK


Número de arquivos sem pré-processamento: 0 


----------------------------------------------------------------------------------------------------
Coleção: Administracao_Universitaria_Mestrado_Profissional - 2 de 112
----------------------------------------------------------------------------------------------------


OK


Número de arquivos sem pré-processamento: 0 


----------------------------------------------------------------------------------------------------
Coleção: Agroecossistemas - 3 de 112
----------------------------------------------------------------------------------------------------


OK


Número de arquivos sem pré-processamento: 0 


---------

In [ ]:
pasta_central = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

qtd_txt_pp = 0

lista_colecoes = sorted([os.path.join(pasta_central,c) for c in os.listdir(pasta_central) if '.' not in c])
qtd_col = len(lista_colecoes)
for cont,colecao in enumerate(lista_colecoes):
  for ano in [os.path.join(colecao,a) for a in os.listdir(colecao) if a.isdigit()]:
    for trabalho in [os.path.join(ano,t) for t in os.listdir(ano) if t.startswith('Trabalho')]:
      if os.path.exists(os.path.join(trabalho,'pre_processamento_c_re.msgpack')):
        qtd_txt_pp += 1
  output.clear()
  print(os.path.basename(colecao),cont+1,'de',qtd_col)
  print(qtd_txt_pp)

output.clear()
print(qtd_txt_pp)

30665


### Ajeitar pastas repetidas de ANOS

In [ ]:
import os
import re
import shutil
!pip install msgpack==1.0.7
import msgpack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.8
    Uninstalling msgpack-1.0.8:
      Successfully uninstalled msgpack-1.0.8


Removendo trabalhos que as cópias são iguais aos originais e depois

Vendo quais pastas ainda estão com cópia

In [ ]:
import re
regex_copia = re.compile(r'\(\d+\)')

pasta_central = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'
for colecao in [os.path.join(pasta_central,c) for c in os.listdir(pasta_central) if '.' not in c]:
  # print(os.path.basename(colecao))
  for ano in [os.path.join(colecao,a) for a in os.listdir(colecao) if '(' in a]:
    for trabalho in [os.path.join(ano,t) for t in os.listdir(ano) if t.startswith('Trabalho')]:
      trabalho_atual = os.path.basename(trabalho)
      ano_atual = os.path.basename(os.path.dirname(trabalho))
      colecao_atual = os.path.basename(os.path.dirname(os.path.dirname(trabalho)))
      ano_atual_sem_copia = regex_copia.sub('', ano_atual).strip()
      print(trabalho_atual,ano_atual,colecao_atual)
    #   caminho_arquivo_copia = os.path.join(trabalho,'pre_processamento_c_re.msgpack')
    #   caminho_arquivo_original = os.path.join(pasta_central,colecao_atual,ano_atual_sem_copia,trabalho_atual,'pre_processamento_c_re.msgpack')
    #   if os.path.exists(caminho_arquivo_copia):
    #     if os.path.exists(caminho_arquivo_original):
    #       pp_copia = abrirArquivoMsgPack(caminho_arquivo_copia)
    #       pp_original = abrirArquivoMsgPack(caminho_arquivo_original)
    #       if pp_original == pp_copia:
    #         print('REMOVENDO ARQUIVO PP DE',trabalho_atual,ano_atual,colecao_atual)
    #         os.remove(caminho_arquivo_copia)
    #     else:
    #       print('NÃO ACHEI ORIGINAL',colecao_atual,ano_atual,trabalho_atual)
    #   else:
    #     print('NÃO ACHEI CÓPIA',colecao_atual,ano_atual,trabalho_atual)
    #   if len(os.listdir(trabalho)) == 0:
    #     print('Removendo',trabalho_atual,'de',ano_atual,'de',colecao_atual)
    #     os.rmdir(trabalho)
    # if len(os.listdir(ano)) == 0:
    #   print('Removendo',ano_atual,'de',colecao_atual)
    #   os.rmdir(ano)
    # print(os.path.basename(ano))



Trabalho 36 2021 (1) Farmacologia_Mestrado_Profissional
Trabalho 175 2008 (1) Neurociencias
Trabalho 33 2020 (1) Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional


Removendo arquivos de pré-processamento que não são iguais, mas que todas as frases da cópia constam no arquivo original

In [ ]:
import re
regex_copia = re.compile(r'\(\d+\)')

pasta_central = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'
for colecao in [os.path.join(pasta_central,c) for c in os.listdir(pasta_central) if '.' not in c]:
  # print(os.path.basename(colecao))
  for ano in [os.path.join(colecao,a) for a in os.listdir(colecao) if '(' in a]:
    for trabalho in [os.path.join(ano,t) for t in os.listdir(ano) if t.startswith('Trabalho')]:
      trabalho_atual = os.path.basename(trabalho)
      ano_atual = os.path.basename(os.path.dirname(trabalho))
      colecao_atual = os.path.basename(os.path.dirname(os.path.dirname(trabalho)))
      ano_atual_sem_copia = regex_copia.sub('', ano_atual).strip()
      print(trabalho_atual,ano_atual,colecao_atual)
      caminho_arquivo_copia = os.path.join(trabalho,'pre_processamento_c_re.msgpack')
      caminho_arquivo_original = os.path.join(pasta_central,colecao_atual,ano_atual_sem_copia,trabalho_atual,'pre_processamento_c_re.msgpack')
      if os.path.exists(caminho_arquivo_copia):
        if os.path.exists(caminho_arquivo_original):
          pp_copia = abrirArquivoMsgPack(caminho_arquivo_copia)
          pp_original = abrirArquivoMsgPack(caminho_arquivo_original)
          copia = True
          for frase_tokenizada in pp_copia:
            if frase_tokenizada not in pp_original:
              copia = False
              break
          if copia:
            print('- REMOVENDO ARQUIVO PP CÓPIA',trabalho_atual,ano_atual,colecao_atual)
            os.remove(caminho_arquivo_copia)
    #       if pp_original == pp_copia:
    #         print('REMOVENDO ARQUIVO PP DE',trabalho_atual,ano_atual,colecao_atual)
    #         os.remove(caminho_arquivo_copia)
    #     else:
    #       print('NÃO ACHEI ORIGINAL',colecao_atual,ano_atual,trabalho_atual)
    #   else:
    #     print('NÃO ACHEI CÓPIA',colecao_atual,ano_atual,trabalho_atual)
      if len(os.listdir(trabalho)) == 0:
        print('Removendo',trabalho_atual,'de',ano_atual,'de',colecao_atual)
        os.rmdir(trabalho)
    if len(os.listdir(ano)) == 0:
      print('Removendo',ano_atual,'de',colecao_atual)
      os.rmdir(ano)
    # print(os.path.basename(ano))

Trabalho 36 2021 (1) Farmacologia_Mestrado_Profissional
- REMOVENDO ARQUIVO PP CÓPIA Trabalho 36 2021 (1) Farmacologia_Mestrado_Profissional
Removendo Trabalho 36 de 2021 (1) de Farmacologia_Mestrado_Profissional
Removendo 2021 (1) de Farmacologia_Mestrado_Profissional
Trabalho 175 2008 (1) Neurociencias
- REMOVENDO ARQUIVO PP CÓPIA Trabalho 175 2008 (1) Neurociencias
Removendo Trabalho 175 de 2008 (1) de Neurociencias
Removendo 2008 (1) de Neurociencias
Trabalho 33 2020 (1) Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional
- REMOVENDO ARQUIVO PP CÓPIA Trabalho 33 2020 (1) Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional
Removendo Trabalho 33 de 2020 (1) de Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional
Removendo 2020 (1) de Propriedade_Intelectual_e_Transferencia_de_Tecnologia_para_Inovacao_Mestrado_Profissional


Última verificação para ver se as pastas copiadas de ano ainda existem

In [ ]:
import re
regex_copia = re.compile(r'\(\d+\)')

pasta_central = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'
for colecao in [os.path.join(pasta_central,c) for c in os.listdir(pasta_central) if '.' not in c]:
  # print(os.path.basename(colecao))
  for ano in [os.path.join(colecao,a) for a in os.listdir(colecao) if '(' in a]:
    print(os.path.basename(ano))